In [233]:
from database.database import Session
from sqlalchemy import select, func
from database.models import Job_1, Job_2, Job_3
from sentence_transformers import SentenceTransformer
import asyncio
import numpy as np 
import pandas as pd

In [234]:
user_data = pd.read_csv("Dataset/User_data_final.csv")

In [235]:
user_data

,URL User,UserID,User Name,Industry,Desired Job,Workplace Desired,Desired Salary,Gender,Marriage,Age,Target,Skills,Degree,Work Experience
0,https://timviec365.vn/ung-vien/nguyen-quan-uv9...,964496,Nguyễn Quân,Vận tải - Lái xe,Nhân viên lái xe bằng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,36,Với kinh nghiệm 10 năm trong lĩnh vực lái xe t...,"- Bằng lái xe hạng C, D, E và có kinh nghiệm 5...",Đơn vị đào tạo: Trường trung cấp nghề,5-10 năm
1,https://timviec365.vn/ung-vien/nguyen-chau-uv9...,986206,Nguyễn Châu,Vận tải - Lái xe,Nhân viên lái xe bằng D,Bình Dương,Thỏa thuận,Nam,Đã có gia đình,44,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trường trung cấp nghề,Trên 10 năm
2,https://timviec365.vn/ung-vien/la-thi-hoang-uv...,769632,LÃ THỊ HOÀNG,Hành chính - Văn phòng,"Nhân viên văn phòng, khu công nghiệp sử dụng t...",Bắc Giang,Thỏa thuận,Nữ,Độc thân,25,- Tìm kiếm một vị trí nhân viên hành chính và ...,- Am hiểu mọi quy trình về hành chính nhân sự ...,Mô tả công việc: CÔNG TY CỔ PHẦN DƯỢC PHẨM AN ...,1-3 năm
3,https://timviec365.vn/ung-vien/nguyen-van-bao-...,981314,Nguyễn Văn Bảo,Vận tải - Lái xe,Nhân viên lái xe hạng C,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,33,Trở thành kỹ sư tốt trong công tác sửa chữa xe...,"Có khả năng chu toàn công việc, siêng năng, tỉ...",Đơn vị đào tạo: Trung tâm dạy nghề,5-10 năm
4,https://timviec365.vn/ung-vien/nguyen-thi-tu-u...,962892,Nguyễn Thị Tứ,Quản lý đơn hàng,Nhân viên thu mua,Hồ Chí Minh,Thỏa thuận,Nữ,Độc thân,24,"- Có một công việc có nhiều cơ hội thăng tiến,...","- Trung thực, cẩn thận, có óc sáng tạo - Kỹ...",Đơn vị đào tạo: Cao Đẳng Kinh Tế Kỹ Thuật Vina...,3-5 năm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,https://timviec365.vn/ung-vien/bui-nghia-uv997...,997433,Bùi Nghĩa,Vận tải - Lái xe,Nhân viên lái xe hạng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,35,"Trở thành nhân viên lái xe cho công ty, tôi mu...","- Tiếp thu tốt các kiến thức lái xe, - Có trí ...",Đơn vị đào tạo: Trường trung cấp nghề,1-3 năm
3979,https://timviec365.vn/ung-vien/tran-thanh-son-...,964381,Trần Thanh Sơn,Vận tải - Lái xe,Nhân viên Lái xe hạng C,Khánh Hòa,Thỏa thuận,Nam,Đã có gia đình,29,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trung học,5-10 năm
3980,https://timviec365.vn/ung-vien/huynh-qui-uv969...,969486,Huỳnh Quí,Vận tải - Lái xe,Nhân viên lái xe bằng B2,Hồ Chí Minh,Thỏa thuận,Nam,Độc thân,20,Áp dụng những kiến thức về giao thông vận tải ...,"Khả năng chịu áp lực cao trong công việc, có t...",Đơn vị đào tạo: Trường trung cấp nghề,Dưới 1 năm
3981,https://timviec365.vn/ung-vien/pham-van-truong...,978744,Phạm Văn Trường,Vận tải - Lái xe,Nhân viên lái xe - Bằng Lái Xe Hạng D,Quảng Ninh,Thỏa thuận,Nam,Đã có gia đình,34,"Đáp ứng tốt các công việc của công ty, giữ gìn...","Giao tiếp tốt, thân thiện, cởi mở với mọi ngườ...",Đơn vị đào tạo: Tốt nghiệp THPT,Trên 10 năm


# Helper Function|

## Khuyến nghị việc làm dựa trên kĩ năng:

In [236]:
async def get_recommendations_based_skill(model, model_name: str, skill: str): 
    # Tính toán embedding cho kỹ năng
    embedding_input = model.encode(skill).tolist()  # Convert tensor/array to list for database compatibility
    
    # Chọn bảng phù hợp dựa trên model_name
    if model_name == "sbert":
        target_table = Job_1
    elif model_name == "phobert":
        target_table = Job_2
    elif model_name == "xlm_roberta":
        target_table = Job_3
    else:
        raise ValueError("Invalid model_name. Supported values are 'sbert', 'phobert' and 'xlm_roberta'.")

    async with Session() as session:
        query = (
            select(
                target_table, 
                target_table.job_skill_vector.cosine_distance(embedding_input).label("distance")
            )
            .where(
                target_table.job_skill != None,
                func.trim(func.regexp_replace(target_table.job_skill, r"[\n\r\u2028]+", "", "g")) != ""
            )
            .order_by("distance")  # Sắp xếp theo khoảng cách cosine
            .limit(20)
        )

        result = await session.execute(query)
        job_results = result.fetchall()

        # Tách job và distance từ kết quả
        jobs = [row[0] for row in job_results]  # `Job_1` or `Job_2` objects
        distances = [row[1] for row in job_results]  # `distance` values
        
        return jobs, distances


## Khuyến nghị việc làm dự trên địa điểm làm việc

In [237]:
async def get_recommendations_based_location(model, model_name: str, location: str):
    # Tính toán embedding cho kỹ năng
    embedding_input = model.encode(location).tolist()  # Convert tensor/array to list for database compatibility

    if model_name == "sbert":
        target_table = Job_1
    elif model_name == "phobert":
        target_table = Job_2
    elif model_name == "xlm_roberta":
        target_table = Job_3
    else:
        raise ValueError("Invalid model_name. Supported values are 'sbert', 'phobert' and 'xlm_roberta'.")
    

    async with Session() as session:
        query = (
            select(target_table,  target_table.job_address_vector.cosine_distance(embedding_input).label("distance"))
            .where(
                target_table.job_address != None,
                func.trim(func.regexp_replace(target_table.job_address, r"[\n\r\u2028]+", "", "g")) != ""
            )
            .order_by("distance")  # Sắp xếp theo khoảng cách cosine
            .limit(20)
        )

        result = await session.execute(query)
        job_results = result.fetchall()

        # Tách job và distance từ kết quả
        jobs = [row[0] for row in job_results]  
        distances = [row[1] for row in job_results]  
        
        return jobs, distances


## Khuyến nghị việc làm dựa trên công việc mơ ước

In [238]:
async def get_recommendations_based_job_title(model, model_name: str, job_title: str):
    # Tính toán embedding cho kỹ năng
    embedding_input = model.encode(job_title).tolist()  # Convert tensor/array to list for database compatibility

    if model_name == "sbert":
        target_table = Job_1
    elif model_name == "phobert":
        target_table = Job_2
    elif model_name == "xlm_roberta":
        target_table = Job_3
    else:
        raise ValueError("Invalid model_name. Supported values are 'sbert', 'phobert' and 'xlm_roberta'.")
    
    async with Session() as session:
        query = (
            select(target_table,  target_table.job_title_vector.cosine_distance(embedding_input).label("distance"))
            .where(
                target_table.job_title != None,
                func.trim(func.regexp_replace(target_table.job_title, r"[\n\r\u2028]+", "", "g")) != ""
            )
            .order_by("distance")  # Sắp xếp theo khoảng cách cosine
            .limit(20)
        )

        result = await session.execute(query)
        job_results = result.fetchall()

        # Tách job và distance từ kết quả
        jobs = [row[0] for row in job_results]  
        distances = [row[1] for row in job_results]  
        
        return jobs, distances


## Khuyến nghị việc làm dựa trên ngành nghề

In [239]:
async def get_recommendations_based_industry(model, model_name: str, industry: str):
    # Tính toán embedding cho kỹ năng
    embedding_input = model.encode(industry).tolist()  # Convert tensor/array to list for database compatibility

    if model_name == "sbert":
        target_table = Job_1
    elif model_name == "phobert":
        target_table = Job_2
    elif model_name == "xlm_roberta":
        target_table = Job_3
    else:
        raise ValueError("Invalid model_name. Supported values are 'sbert', 'phobert' and 'xlm_roberta'.")
    
    async with Session() as session:
        query = (
            select(target_table,  target_table.industry_vector.cosine_distance(embedding_input).label("distance"))
            .where(
                target_table.industry != None,
                func.trim(func.regexp_replace(target_table.industry, r"[\n\r\u2028]+", "", "g")) != ""
            )
            .order_by("distance")  # Sắp xếp theo khoảng cách cosine
            .limit(20)
        )

        result = await session.execute(query)
        job_results = result.fetchall()

        # Tách job và distance từ kết quả
        jobs = [row[0] for row in job_results]  
        distances = [row[1] for row in job_results]  
        
        return jobs, distances


## Khuyến nghị dựa trên cả 4 yếu tố:

In [240]:
async def get_recommendations_based_combined(model, model_name: str, industry: str, job_title: str, job_skill: str, job_address: str):
    # Tính toán embedding cho kỹ năng
    industry_input = model.encode(industry).tolist()  # Convert tensor/array to list for database compatibility
    job_title_input = model.encode(job_title).tolist()
    job_skill_input = model.encode(job_skill).tolist()
    job_address_input = model.encode(job_address).tolist()

    if model_name == "sbert":
        target_table = Job_1
    elif model_name == "phobert":
        target_table = Job_2
    elif model_name == "xlm_roberta":
        target_table = Job_3
    else:
        raise ValueError("Invalid model_name. Supported values are 'sbert', 'phobert' and 'xlm_roberta'.")

    async with Session() as session:
        query = (
            select(target_table,  
                   target_table.job_title_vector.cosine_distance(job_title_input).label("job_title_distance"),
                   target_table.industry_vector.cosine_distance(industry_input).label("industry_distance"),
                   target_table.job_skill_vector.cosine_distance(job_skill_input).label("job_skill_distance"),
                   target_table.job_address_vector.cosine_distance(job_address_input).label("job_address_distance")
            )
            .where(
                target_table.industry != None,
                target_table.job_title != None,
                target_table.job_skill != None,
                target_table.job_address != None,
                func.trim(func.regexp_replace(target_table.industry, r"[\n\r\u2028]+", "", "g")) != "",
                func.trim(func.regexp_replace(target_table.job_title, r"[\n\r\u2028]+", "", "g")) != "",
                func.trim(func.regexp_replace(target_table.job_skill, r"[\n\r\u2028]+", "", "g")) != "",
                func.trim(func.regexp_replace(target_table.job_address, r"[\n\r\u2028]+", "", "g")) != "",
            )
            .order_by("job_title_distance", "industry_distance",  "job_skill_distance", "job_address_distance")  # Sắp xếp theo khoảng cách cosine
            .limit(20)
        )

        result = await session.execute(query)
        job_results = result.fetchall()

        # Tách job và distance từ kết quả
        jobs = [row[0] for row in job_results]  # `Job_1` objects
        industry_distances = [row[1] for row in job_results]  # `distance` values
        job_title_distances = [row[2] for row in job_results]
        job_skill_distances = [row[3] for row in job_results]
        job_address_distances = [row[4] for row in job_results]
        
        return jobs, industry_distances, job_title_distances, job_skill_distances, job_address_distances


In [241]:
async def Get_multi_recommendations(df, model, model_name: str, method: str):
    """
    Mở rộng thông tin gợi ý công việc với khoảng cách cho mỗi người trong DataFrame.
    Dựa trên phương thức ('industry' hoặc 'skill'), sẽ gọi hàm thích hợp.

    Args:
        df (pd.DataFrame): DataFrame chứa thông tin người dùng.
        model: Mô hình để tính toán embedding ngành nghề hoặc kỹ năng.
        model_name (str): Tên của mô hình (e.g., "sbert", "phobert").
        method (str): Phương thức gọi hàm (có thể là 'industry' hoặc 'skill').

    Returns:
        pd.DataFrame: DataFrame mở rộng với các công việc gợi ý và khoảng cách cho từng người.
    """
    # Giới hạn dataframe chỉ lấy 10 người đầu tiên
    df = df.sample(n=10, random_state=42)

    # Danh sách kết quả mở rộng
    expanded_results = []

    # Duyệt qua từng người dùng trong dataframe
    for _, row in df.iterrows():
        # Lấy thông tin ngành nghề
        
        try:
            # Gọi hàm gợi ý công việc dựa trên phương thức (industry hoặc skill)
            if method == "industry":
                jobs, distances = await get_recommendations_based_industry(model, model_name, row["Industry"])
            elif method == "skill":
                jobs, distances = await get_recommendations_based_skill(model, model_name, row["Skills"])
            elif method == "location":
                jobs, distances = await get_recommendations_based_location(model, model_name, row["Workplace Desired"])
            elif method == "job title":
                jobs, distances = await get_recommendations_based_job_title(model, model_name, row["Desired Job"])
            elif method == "combined":
                jobs, distances, job_title_distances, job_skill_distances, job_address_distances = await get_recommendations_based_combined(model, model_name, row["Industry"], row["Desired Job"], row["Skills"], row["Workplace Desired"])
            else:
                raise ValueError("Method must be either 'industry' or 'skill'.")

            # Tạo một dòng riêng biệt cho mỗi công việc gợi ý
            for job, distance in zip(jobs, distances):
                expanded_results.append({
                    "User ID": row.get("UserID", None),
                    "User Name": row.get("User Name", None),
                    "Industry": row.get("Industry", None),
                    "Age": row.get("Age", None),
                    "Job Title Recommended": job.job_title,
                    "Company Name": job.company_name,
                    "Industry Recommended": job.industry,
                    "Skills Recommended": job.job_skill,
                    "Address Recommended": job.job_address,
                    "Job Requierements": job.job_requirements,
                    "Job Description": job.job_description,
                    "Job Salary": job.salary,
                    "Job Years of Experience": job.years_of_experience,
                    "Distance": distance
                })
        except Exception as e:
            print(e)

    # Chuyển kết quả mở rộng thành dataframe
    expanded_df = pd.DataFrame(expanded_results)
    return expanded_df


# Models

## sBERT

In [242]:
model = SentenceTransformer("keepitreal/vietnamese-sbert")

In [243]:
skill_recommendation_results = await Get_multi_recommendations(user_data, model, "sbert", "skill")
skill_recommendation_results

2024-12-16 01:27:41,650 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:27:41,655 INFO sqlalchemy.engine.Engine SELECT jobs_1.id, jobs_1.url, jobs_1.job_title, jobs_1.company_name, jobs_1.industry, jobs_1.company_address, jobs_1.job_description, jobs_1.job_requirements, jobs_1.benefits, jobs_1.salary, jobs_1.expire_date, jobs_1.employee_rank, jobs_1.job_skill, jobs_1.job_field, jobs_1.years_of_experience, jobs_1.job_address, jobs_1.job_title_vector, jobs_1.industry_vector, jobs_1.job_skill_vector, jobs_1.job_address_vector, jobs_1.job_skill_vector <=> $1 AS distance 
FROM jobs_1 
WHERE jobs_1.job_skill IS NOT NULL AND trim(regexp_replace(jobs_1.job_skill, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:27:41,657 INFO sqlalchemy.engine.Engine [cached since 7549s ago] ('[0.03798454627394676,0.3454383909702301,0.017232272773981094,0.015160647220909595,0.2480212152004242,0.8499230146408081,0.041743144392967224,-0.11242

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Phiên Dịch Tiếng Nhật,Công Ty TNHH YKK Việt Nam,Dệt may/May mặc/Giày dép,"Kỹ năng giao tiêp, Kỹ năng Tin học văn phòng, ...","KCN Đồng Văn 3, Phường Hoàng Đông, Thị xã Duy ...",1. Bằng cấp: Tốt nghiệp Cao đẳng trở lên (chuy...,"MÔ TẢ CÔNG VIỆC. Làm các thủ tục, giấy tờ cho ...",Thương lượng,1-3 năm,0.312839
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Nhóm Sự Kiện,Hệ thống Giáo Dục Edufit,Giáo dục/Đào Tạo,"Kỹ năng giao tiếp, Kỹ năng giao tiếp, Kỹ năng ...","lô PT-01, KĐT Vinhomes Ocean Park, Đa Tốn, Gia...",Tốt nghiệp đại học trở lên. Khả năng sử dụng t...,Tổ chức/Thực hiện/Giám sát/Quản lý công việc S...,"Trên 40,000,000",1-3 năm,0.355655
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Đào Tạo,Công Ty Vàng Bạc Đá Quý Bảo Tín Minh Châu,Bán lẻ/Bán sỉ,"Kỹ năng giao tiếp, Kỹ năng lập kế hoạch, Soạn ...",139 Cầu Giấy,"Giới tính: Nam và Nữ, độ tuổi từ 35 – 45 tuổi;...",I. MÔ TẢ CÔNG VIỆC:. Phụ trách tổ chức vận hàn...,"20,000,000 - 40,000,000",3-5 năm,0.365367
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Nhóm Chất Lượng (QA),Alphanam Group,Kỹ thuật xây dựng/Cơ sở hạ tầng,"Kỹ năng giao tiếp, Kỹ năng giao tiếp, Kỹ năng ...","108 Nguyễn Trãi, Thượng Đình, Thanh Xuân, Hà N...",Tốt nghiệp Đại học trở lên các chuyên ngành: Q...,* Xây dựng hệ thống văn bản chất lượng (Quy tr...,Thương lượng,5-10 năm,0.414016
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Bộ phận Mua Hàng,Công Ty Cổ Phần Ánh Dương Building (ADB),Vật liệu xây dựng,"Kỹ năng giao tiếp, Kỹ năng giao tiếp, Làm việc...","Cao ốc An Khang Building, Số 28, Đường số 19, ...",2. Yêu cầu tuyển dụng:. 2.1. Trình độ:. Nam/nữ...,1. Trách nhiệm công việc:. Xây dựng kế hoạch m...,"Trên 40,000,000",3-5 năm,0.420976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Kế Toán Tổng Hợp,Cty TNHH Quốc Tế Anh Duy,Nhập khẩu/Xuất khẩu,"Lập Báo Cáo Tài Chính, Kế Toán Tổng Hợp, Phần ...","170 Ung Văn Khiêm, Phường 25, Bình Thạnh, Thàn...",Yêu cầu: Nữ; Tốt nghiệp Đại Học/Cao đẳng. Thàn...,Kiểm tra đối chiếu số liệu giữa các đơn vị nội...,Thương lượng,1-3 năm,0.458464
196,973260,Lê Thanh,Vận tải - Lái xe,24,Quản Lý Điều Hành Doanh Nghiệp (Operations Man...,CÔNG TY TNHH SMART OUTSOURCING,Kế toán/Kiểm toán,"Lập Kế Hoạch Tài Chính, MBA, Quản Lý Điều Hành...","Tầng 9, Tòa văn phòng C1 Thành Công, phường Th...",III. YÊU CẦU. Tốt nghiệp trình độ Đại họ...,I . KẾT QUẢ CÔNG VIỆC CẦN ĐẠT. Giám sát và quả...,"15,000,000 - 20,000,000",1-3 năm,0.459906
197,973260,Lê Thanh,Vận tải - Lái xe,24,Kế Toán Nội Bộ,Công Ty Cổ Phần Vận Tải Biển Container Thái Bì...,Hậu cần/Giao nhận,"Kế Toán, Kế Toán Nội Bộ, Báo Cáo Tài Chính, Ph...","Tầng 15 tòa nhà Hòa Bình, 106 Hoàng Quốc Việt,...",YÊU CẦU CÔNG VIỆC:. + Tốt nghiệp ĐH chuyên ngà...,MÔ TẢ CÔNG VIỆC:. Nhập dữ liệu định khoản lên ...,"10,000,000 - 15,000,000",3-5 năm,0.460947
198,973260,Lê Thanh,Vận tải - Lái xe,24,Xưởng Trưởng Bảo Dưỡng Sửa Chữa Ô Tô - Hồ Chí ...,Công Ty TNHH Dịch Vụ Vận Tải Sinh Thái Vinbus,Ô tô,"Quản Trị Nhân Sự, Sửa Chữa Bảo Dưỡng, Điều Hàn...","KĐT Vinhomes Grand Park, Số 10 đường T10 phân ...","Bằng cấp chuyên môn. Tốt nghiệp Đại học, Cao đ...","Chịu trách nhiệm toàn bộ về công tác tổ chức, ...","Trên 40,000,000",3-5 năm,0.463100


In [244]:
address_recommendation_results = await Get_multi_recommendations(user_data, model, "sbert", "location")
address_recommendation_results

2024-12-16 01:27:56,755 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:27:56,759 INFO sqlalchemy.engine.Engine SELECT jobs_1.id, jobs_1.url, jobs_1.job_title, jobs_1.company_name, jobs_1.industry, jobs_1.company_address, jobs_1.job_description, jobs_1.job_requirements, jobs_1.benefits, jobs_1.salary, jobs_1.expire_date, jobs_1.employee_rank, jobs_1.job_skill, jobs_1.job_field, jobs_1.years_of_experience, jobs_1.job_address, jobs_1.job_title_vector, jobs_1.industry_vector, jobs_1.job_skill_vector, jobs_1.job_address_vector, jobs_1.job_address_vector <=> $1 AS distance 
FROM jobs_1 
WHERE jobs_1.job_address IS NOT NULL AND trim(regexp_replace(jobs_1.job_address, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:27:56,761 INFO sqlalchemy.engine.Engine [cached since 5925s ago] ('[-0.2742690145969391,0.4564611315727234,-0.6578325629234314,-0.35101208090782166,-0.26071643829345703,-0.12473781406879425,0.3879827558994293,-

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Phó Quản Lý Chi Nhánh Thẩm Mỹ,Công Ty TNHH Dược Mỹ Phẩm H-Derma,Sản xuất/Vận hành sản xuất,"Mỹ Phẩm, Giám Sát Bán Hàng, Quản Lý Điều Hành",Khánh Hòa,Tối thiểu trên 01 năm kinh nghiệm làm vị trí t...,"Xây dựng, tổ chức, vận hành giám sát công việc...","20,000,000 - 40,000,000",Không yêu cầu kinh nghiệm,0.000000
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Phụ Trách Kinh Doanh,Công Ty TNHH Thép Tân Thắng Lợi,Kinh doanh,"Tư Vấn Bán Hàng, Phát Triển Thị Trường",Khánh Hòa,Tốt nghiệp cao đẳng trở lên chuyên ngành Thươn...,"Xây dựng, tìm các đối tác khách hàngBáo giá kh...","15,000,000 - 20,000,000",Không yêu cầu kinh nghiệm,0.000000
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Vận Hành Dịch Vụ,Công Ty Cổ Phần Phần Dịch Vụ Bạn Uống Tôi Lái,Kinh doanh,"Quản Trị Hệ Thống, Hành chính Văn phòng","|,Khánh Hòa","Yêu cầu ứng viên:Tốt nghiệp trung cấp, cao đẳn...",Chủ động liên hệ với khách hàng để giải quyết ...,"5,000,000 - 10,000,000",Không yêu cầu kinh nghiệm,0.144465
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Bán Hàng Vùng Kênh Truyền Thống - Area ...,Công Ty Cổ Phần Sữa Việt Nam (Vinamilk),Hàng tiêu dùng,"Direct Sales, General Trade","Khánh Hòa, Vietnam",HỌC VẤN & CHUYÊN NGÀNH:. Tốt nghiệp đại học. Ư...,KHU VỰC PHỤ TRÁCH: Các tỉnh từ Khánh Hòa vào đ...,Thương lượng,5-10 năm,0.201885
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,New Product Introduction Engineer - NPI Engineer,Onsemi Viet Nam,Sản xuất,"Electronics, New Product Qualification, Lean S...",Biên Hòa,"Qualifications. BS/MS in Electrical, Mechanica...",Job Description. Introduces and maintains asse...,Thương lượng,Không yêu cầu kinh nghiệm,0.260227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Kỹ Thuật IT Sửa PC Laptop Máy In,Công Ty TNHH Phát Triển Công Nghệ Trường Thịnh...,"Công nghệ thông tin, Quảng cáo/Marketing/PR","IT Helpdesk, IT Support",Hồ Chí Minh,Có Ít nhất > 1 năm KN làm chuyên viên kỹ thuật...,"Mua bán sửa chữa máy tính bàn pc, laptop, máy ...","5,000,000 - 10,000,000",1-3 năm,0.000000
196,973260,Lê Thanh,Vận tải - Lái xe,24,- Nhân Viên Phụ Bếp Part Time,Công ty Cổ phần Đầu tư Thương mại Quốc tế Mặt ...,Thực phẩm/DV ăn uống,"Phụ Bếp, Chế Biến Thực Phẩm, Tư Vấn Bán Hàng",Hồ Chí Minh,Yêu Cầu Công ViệcLàm việc tại: Nhà hàng Hotpot...,Thực hiện sơ chế các nguyên vật liệu dùng đ...,Thỏa thuận,Không yêu cầu kinh nghiệm,0.000000
197,973260,Lê Thanh,Vận tải - Lái xe,24,Quản Lý Kinh Doanh Thu Nhập Từ 20TR,Công Ty TNHH Bảo Hiểm Nhân Thọ Cathay - Việt Nam,bảo hiểm,"Chăm Sóc Khách Hàng, Tư Vấn Bán Hàng, Tư Vấn B...",Hồ Chí Minh,Tốt nghiệp trung cấp trở lên trên 26 tuổi. Đã ...,"Quản lý, đào tạo nhân viên, hỗ trợ tuyển dụng....","20,000,000 - 40,000,000",Không yêu cầu kinh nghiệm,0.000000
198,973260,Lê Thanh,Vận tải - Lái xe,24,Store Manager - Cửa Hàng Trưởng,Công ty TNHH Manpower Việt Nam - Hồ Chí Minh,Nhân Sự,"Quản Lý Điều Hành, Giám Sát Bán Hàng, Bán Lẻ",Hồ Chí Minh,At least 1 year of experience in fashion retai...,Stock & Sales Management:. Ensure effective re...,"20,000,000 - 40,000,000",1-3 năm,0.000000


In [245]:
industry_recommendation_results = await Get_multi_recommendations(user_data, model, "sbert", "industry")
industry_recommendation_results

2024-12-16 01:28:01,419 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:28:01,425 INFO sqlalchemy.engine.Engine SELECT jobs_1.id, jobs_1.url, jobs_1.job_title, jobs_1.company_name, jobs_1.industry, jobs_1.company_address, jobs_1.job_description, jobs_1.job_requirements, jobs_1.benefits, jobs_1.salary, jobs_1.expire_date, jobs_1.employee_rank, jobs_1.job_skill, jobs_1.job_field, jobs_1.years_of_experience, jobs_1.job_address, jobs_1.job_title_vector, jobs_1.industry_vector, jobs_1.job_skill_vector, jobs_1.job_address_vector, jobs_1.industry_vector <=> $1 AS distance 
FROM jobs_1 
WHERE jobs_1.industry IS NOT NULL AND trim(regexp_replace(jobs_1.industry, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:28:01,427 INFO sqlalchemy.engine.Engine [cached since 2204s ago] ('[0.44310906529426575,0.04457218572497368,-0.14286266267299652,-0.24837009608745575,-0.1243777871131897,-0.26507705450057983,-0.1482895314693451,-0.58 ..

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Digital Marketing Hỗ Trợ Vận Hành Sà...,Công Ty Cổ Phần Thương Mại Thiết Bị Văn Phòng ...,Trang thiết bị văn phòng,"Social Marketing, Digital Marketing, Truyền Th...","43 Linh Đông, Khu Phố 7, P. Linh Đông, Tp Thủ Đức","Tốt nghiệp Cao đẳng chuyên ngành Marketing, bá...","Lập kế hoạch, phối hợp tổ chức và triển khai c...",Thỏa thuận,1-3 năm,0.430879
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Tư Vấn Văn Phòng (Thu Nhập Đến 20 Tr...,Công Ty TNHH Một Thành Viên Đức Lan,Trang thiết bị văn phòng,"Tư Vấn Doanh Nghiệp (B2B), Tư Vấn Bán Hàng, Bá...","số 149 Võ Văn Tần, Võ Thị Sáu, Quận 3, Hồ Chí ...","Nữ, Tốt nghiệp trung cấp trở lênƯu tiên có kin...","Công ty chuyên phân phối và cung cấp Máy, Mực,...","20,000,000 - 40,000,000",Không yêu cầu kinh nghiệm,0.430879
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Phó Bộ Phận QC,Công Ty Cổ Phần Văn Phòng Phẩm Hải Phòng,Trang thiết bị văn phòng,"Quản Lý Chất Lượng (QA/QC), Quản Lý Sản Xuất",Tổ 1 – KDC Phương Lung – Đường Mạc Đăng Doanh ...,Nam/Nữ từ 25 40 tuổi;. Tốt nghiệp Đại học các ...,"Quản lý, phân công công việc cho nhân viên;. K...",Thỏa thuận,1-3 năm,0.430879
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Kinh Doanh (Thu Nhập Đến 20 Triệu),Công Ty TNHH Một Thành Viên Đức Lan,Trang thiết bị văn phòng,"Tư Vấn Doanh Nghiệp (B2B), Tư Vấn Bán Hàng, Bá...","số 149 Võ Văn Tần, Võ Thị Sáu, Quận 3, Hồ Chí ...","Nữ, Tốt nghiệp trung cấp trở lênƯu tiên có kin...","Công ty chuyên phân phối và cung cấp Máy, Mực,...","20,000,000 - 40,000,000",Không yêu cầu kinh nghiệm,0.430879
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Lập Trình Mobile (Flutter Dev),Công Ty TNHH Công Nghệ Katec,"Kinh doanh, Trang thiết bị văn phòng","Android, iOS, Lập trình Mobile, UI/UX","189 PHAN HUY CHÚ, AN KHÁNH, NINH KIỀU, TP CT,C...",Có ít nhất 1 năm kinh nghiệm làm việc về phát ...,Tham gia phát triển các dự án của công ty.Xây ...,"5,000,000 - 10,000,000",1-3 năm,0.444103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Customs Clearance Manager,Samsung SDS,Vận tải,Electronics,"SEHC, Cổng số 1, Tăng Nhơn Phú B, Quận 9, Thàn...",(1)EDUCATION / QUALIFICATION:. College or univ...,(1) BROAD SCOPE:. Consulting of customs cleara...,"Trên 40,000,000",Trên 10 năm,0.221493
196,973260,Lê Thanh,Vận tải - Lái xe,24,Account Executive,"FedEx Express Vietnam Co., Ltd.",Vận tải,"Communications, Negotiation","15F, 4 Ton That Tung Street, Trung Tu Ward, Do...","To do the job well, you will need the followin...","In the position of an Account Executive, your ...",Thương lượng,1-3 năm,0.221493
197,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Khai Thác Sân Bay Nội Bài Không Yêu ...,Công Ty Cổ Phần Nhà Ga Hàng Hóa ALS,Vận tải,"Quản Lý Kho, Xuất Nhập Khẩu, Giao Nhận Kho Vận...","Tầng 4, ga hàng hóa ALS, Cảng HKQT Nội bài, Ph...",Trình độ học vấn: Tốt nghiệp các trường Đại họ...,* Nhân viên sẽ phụ trách làm một số mảng công ...,"Trên 40,000,000",Không yêu cầu kinh nghiệm,0.221493
198,973260,Lê Thanh,Vận tải - Lái xe,24,Hà Nội - Chuyên Viên Kinh Doanh (Bán Xe Ô Tô V...,GSM,Vận tải,"Kinh doanh ô tô, Sales, Marketing","Tầng B1 tòa R3 Royal City, 72 Nguyễn Trãi, Tha...",Tốt nghiệp từ Trung cấp trở lên. Ít nhất 1 năm...,"Tìm kiếm Khách hàng, triển khai bán hàng, hoàn...","Trên 40,000,000",Không yêu cầu kinh nghiệm,0.221493


In [246]:
job_title_recommendation_results = await Get_multi_recommendations(user_data, model, "sbert", "job title")
job_title_recommendation_results

2024-12-16 01:28:05,361 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:28:05,363 INFO sqlalchemy.engine.Engine SELECT jobs_1.id, jobs_1.url, jobs_1.job_title, jobs_1.company_name, jobs_1.industry, jobs_1.company_address, jobs_1.job_description, jobs_1.job_requirements, jobs_1.benefits, jobs_1.salary, jobs_1.expire_date, jobs_1.employee_rank, jobs_1.job_skill, jobs_1.job_field, jobs_1.years_of_experience, jobs_1.job_address, jobs_1.job_title_vector, jobs_1.industry_vector, jobs_1.job_skill_vector, jobs_1.job_address_vector, jobs_1.job_title_vector <=> $1 AS distance 
FROM jobs_1 
WHERE jobs_1.job_title IS NOT NULL AND trim(regexp_replace(jobs_1.job_title, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:28:05,364 INFO sqlalchemy.engine.Engine [cached since 6098s ago] ('[0.34603989124298096,0.04691627249121666,-0.0481988899409771,-0.07198398560285568,-0.26271557807922363,-0.096248559653759,-0.023860599845647812,-0.15

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính - Nhân Sự,Viện Nghiên Cứu Và Đào Tạo Bất Động Sản,Kinh doanh,"Đào Tạo, Quản Lý Điều Hành, Quản Trị Nhân Sự (...","Đà Nẵng: 136 Tống Phước Phổ, Hải Châu",Tốt nghiệp Đại học chuyên ngành Kinh tế/ Quản ...,Thực hiện công tác phát triển nhân sự.Triển kh...,Thỏa thuận,1-3 năm,0.179899
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính - Nhân Sự,Công Ty TNHH Thương Mại Và Dịch Vụ Tocepo,Kinh doanh,Hành chính Văn phòng,"Số 224 Đống Đa, P. Thị Nại, TP. Qui Nhơn, tỉnh...",Có bằng Cử nhân trở lên thuộc chuyên ngành Quả...,"Quản lý hệ thống các chính sách, quy định, quy...",Thỏa thuận,1-3 năm,0.179899
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính Nhân Sự,Công Ty CP Thực Phẩm Richy Miền Nam,Sản xuất,"Tuyển Dụng, Quản Trị Nhân Sự, Nhân Sự, Hành Ch...","Tầng 8, Tòa Victory Tower, Số 12 Tân Trào, P T...",Tốt nghiệp Đại học trở lên. Ứng viên 03 năm ki...,1. Nhân sự:. a. Xây dựng và quản trị hệ thống:...,"20,000,000 - 40,000,000",3-5 năm,0.188543
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính Nhân Sự,Công Ty TNHH Thời Trang Tracy,Kinh doanh,Hành chính Văn phòng,"141 Bà Triệu, Hai Bà Trưng",●Giới tính: Nữ. ●Độ tuổi: Sinh từ 1985 đến 199...,Công tác hành chính. ● Quản lý và phê duyệt cô...,Thỏa thuận,5-10 năm,0.188543
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính Nhân Sự,Công Ty TNHH Kim Hà Việt,Sản xuất,"Quản Trị Nhân Sự, HR, Tuyển Dụng, Hành Chính N...",Tra Vinh,Tốt nghiệp đại học các chuyên ngành Quản trị N...,"1. Công tác nhân sự:. Tổ chức, quản lý công tá...","Trên 40,000,000",5-10 năm,0.188543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,QA Executive (Ưu Tiên Có Bằng Lái Xe B2),Rentokil Initial (Viet Nam) Limited Company,Dịch vụ môi trường/Chất thải,"Quality Assurance, Quality Control, Pest Contr...","P. Hưng Phú, Quận Cái Răng, TP. Cần Thơ","Graduated from college and above, chemical or ...",1. Inspection & Reporting. To constantly monit...,"Trên 40,000,000",1-3 năm,0.482940
196,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Lái Xe Sân Đỗ,Công ty TNHH Một thành viên Kỹ thuật Máy bay -...,Cơ khí/Kĩ thuật ứng dụng,"Lái Xe, Lái Xe Tải","Sân bay quốc tế Nội Bài, Sóc Sơn, Hà Nội|",Là công dân Việt Nam; từ đủ 18 tuổi đến 45 tuổ...,Lái xe trên sân đỗ máy bay phục vụ công tác bả...,Thỏa thuận,Không yêu cầu kinh nghiệm,0.492187
197,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Bán Hàng Mercedes-Benz Khu Vực Hà Nội,Mercedes-Benz Haxaco Láng Hạ,Ô tô,"Kinh doanh ô tô, Bán Hàng Cao Cấp, Ô Tô, Bán H...","46 Phố Láng Hạ, phường Láng Hạ, Đống Đa, Hà Nộ...",Kinh nghiệm: Có từ 12 năm kinh nghiệm bán hàng...,Chúng tôi tìm kiếm nhân sự vàng cho vị trí Nhâ...,"Trên 40,000,000",Không yêu cầu kinh nghiệm,0.500344
198,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Lái Xe Bằng Lái E,Công Ty Du Lịch Hưng Long,Du lịch,"Lái Xe, Lái Xe Khách","338 Trần Khát Chân, Thanh Nhàn, Hai Bà Trưng, ...",Bằng lái: E ( ưu tiên có kinh nghiệm lái xe kh...,"Phục vụ, chăn sóc khách hàng trong suốt chuyến...","20,000,000 - 40,000,000",Không yêu cầu kinh nghiệm,0.501657


In [247]:
combined_recommendation_results = await Get_multi_recommendations(user_data, model, "sbert", "combined")
combined_recommendation_results

2024-12-16 01:28:08,450 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:28:08,455 INFO sqlalchemy.engine.Engine SELECT jobs_1.id, jobs_1.url, jobs_1.job_title, jobs_1.company_name, jobs_1.industry, jobs_1.company_address, jobs_1.job_description, jobs_1.job_requirements, jobs_1.benefits, jobs_1.salary, jobs_1.expire_date, jobs_1.employee_rank, jobs_1.job_skill, jobs_1.job_field, jobs_1.years_of_experience, jobs_1.job_address, jobs_1.job_title_vector, jobs_1.industry_vector, jobs_1.job_skill_vector, jobs_1.job_address_vector, jobs_1.job_title_vector <=> $1 AS job_title_distance, jobs_1.industry_vector <=> $2 AS industry_distance, jobs_1.job_skill_vector <=> $3 AS job_skill_distance, jobs_1.job_address_vector <=> $4 AS job_address_distance 
FROM jobs_1 
WHERE jobs_1.industry IS NOT NULL AND jobs_1.job_title IS NOT NULL AND jobs_1.job_skill IS NOT NULL AND jobs_1.job_address IS NOT NULL AND trim(regexp_replace(jobs_1.industry, $5::VARCHAR, $6::VARCHAR, $7::VARCHAR)) != $8::VAR

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính - Nhân Sự,Viện Nghiên Cứu Và Đào Tạo Bất Động Sản,Kinh doanh,"Đào Tạo, Quản Lý Điều Hành, Quản Trị Nhân Sự (...","Đà Nẵng: 136 Tống Phước Phổ, Hải Châu",Tốt nghiệp Đại học chuyên ngành Kinh tế/ Quản ...,Thực hiện công tác phát triển nhân sự.Triển kh...,Thỏa thuận,1-3 năm,0.179899
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính - Nhân Sự,Công Ty TNHH Thương Mại Và Dịch Vụ Tocepo,Kinh doanh,Hành chính Văn phòng,"Số 224 Đống Đa, P. Thị Nại, TP. Qui Nhơn, tỉnh...",Có bằng Cử nhân trở lên thuộc chuyên ngành Quả...,"Quản lý hệ thống các chính sách, quy định, quy...",Thỏa thuận,1-3 năm,0.179899
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính Nhân Sự,Công Ty TNHH Kim Hà Việt,Sản xuất,"Quản Trị Nhân Sự, HR, Tuyển Dụng, Hành Chính N...",Tra Vinh,Tốt nghiệp đại học các chuyên ngành Quản trị N...,"1. Công tác nhân sự:. Tổ chức, quản lý công tá...","Trên 40,000,000",5-10 năm,0.188543
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính Nhân Sự,Công Ty CP Thực Phẩm Richy Miền Nam,Sản xuất,"Tuyển Dụng, Quản Trị Nhân Sự, Nhân Sự, Hành Ch...","Tầng 8, Tòa Victory Tower, Số 12 Tân Trào, P T...",Tốt nghiệp Đại học trở lên. Ứng viên 03 năm ki...,1. Nhân sự:. a. Xây dựng và quản trị hệ thống:...,"20,000,000 - 40,000,000",3-5 năm,0.188543
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính Nhân Sự,Công Ty Cổ Phần Hóa Dầu Vĩnh Phúc,Kinh doanh,"Quản Trị Nhân Sự (HR), Bảo Hiểm Và Tiền Lương ...","10A/13, QL1A, Tân Hưng, Tân Hạnh, Long Hồ, Vĩn...",· Có bằng cử nhân trở lên chuyên ngành Kinh tế...,· Xây dựng kế hoạch tuyển dụng. · Phối hợp các...,"10,000,000 - 15,000,000",1-3 năm,0.188543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,QA Executive (Ưu Tiên Có Bằng Lái Xe B2),Rentokil Initial (Viet Nam) Limited Company,Dịch vụ môi trường/Chất thải,"Quality Assurance, Quality Control, Pest Contr...","P. Hưng Phú, Quận Cái Răng, TP. Cần Thơ","Graduated from college and above, chemical or ...",1. Inspection & Reporting. To constantly monit...,"Trên 40,000,000",1-3 năm,0.482940
196,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Lái Xe Sân Đỗ,Công ty TNHH Một thành viên Kỹ thuật Máy bay -...,Cơ khí/Kĩ thuật ứng dụng,"Lái Xe, Lái Xe Tải","Sân bay quốc tế Nội Bài, Sóc Sơn, Hà Nội|",Là công dân Việt Nam; từ đủ 18 tuổi đến 45 tuổ...,Lái xe trên sân đỗ máy bay phục vụ công tác bả...,Thỏa thuận,Không yêu cầu kinh nghiệm,0.492187
197,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Bán Hàng Mercedes-Benz Khu Vực Hà Nội,Mercedes-Benz Haxaco Láng Hạ,Ô tô,"Kinh doanh ô tô, Bán Hàng Cao Cấp, Ô Tô, Bán H...","46 Phố Láng Hạ, phường Láng Hạ, Đống Đa, Hà Nộ...",Kinh nghiệm: Có từ 12 năm kinh nghiệm bán hàng...,Chúng tôi tìm kiếm nhân sự vàng cho vị trí Nhâ...,"Trên 40,000,000",Không yêu cầu kinh nghiệm,0.500344
198,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Lái Xe Bằng Lái E,Công Ty Du Lịch Hưng Long,Du lịch,"Lái Xe, Lái Xe Khách","338 Trần Khát Chân, Thanh Nhàn, Hai Bà Trưng, ...",Bằng lái: E ( ưu tiên có kinh nghiệm lái xe kh...,"Phục vụ, chăn sóc khách hàng trong suốt chuyến...","20,000,000 - 40,000,000",Không yêu cầu kinh nghiệm,0.501657


In [248]:
industry_recommendation_results.to_csv("results/sbert/industry_recommendation_results.csv", index=False)
skill_recommendation_results.to_csv("results/sbert/skill_recommendation_results.csv", index=False)
address_recommendation_results.to_csv("results/sbert/address_recommendation_results.csv", index=False)
job_title_recommendation_results.to_csv("results/sbert/job_title_recommendation_results.csv", index=False)
combined_recommendation_results.to_csv("results/sbert/combined_recommendation_results.csv", index=False)

## PhoBERT

In [249]:
model = SentenceTransformer('vinai/phobert-base')

No sentence-transformers model found with name vinai/phobert-base. Creating a new one with mean pooling.


In [250]:
skill_recommendation_results = await Get_multi_recommendations(user_data, model, "phobert", "skill")
skill_recommendation_results

2024-12-16 01:28:20,722 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:28:20,730 INFO sqlalchemy.engine.Engine SELECT jobs_2.id, jobs_2.url, jobs_2.job_title, jobs_2.company_name, jobs_2.industry, jobs_2.company_address, jobs_2.job_description, jobs_2.job_requirements, jobs_2.benefits, jobs_2.salary, jobs_2.expire_date, jobs_2.employee_rank, jobs_2.job_skill, jobs_2.job_field, jobs_2.years_of_experience, jobs_2.job_address, jobs_2.job_title_vector, jobs_2.industry_vector, jobs_2.job_skill_vector, jobs_2.job_address_vector, jobs_2.job_skill_vector <=> $1 AS distance 
FROM jobs_2 
WHERE jobs_2.job_skill IS NOT NULL AND trim(regexp_replace(jobs_2.job_skill, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:28:20,732 INFO sqlalchemy.engine.Engine [cached since 1121s ago] ('[-0.13463221490383148,0.3770056366920471,-0.14832065999507904,-0.20196151733398438,-0.06632264703512192,0.2360406368970871,0.16971825063228607,-0.184

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Nhóm Sự Kiện,Hệ thống Giáo Dục Edufit,Giáo dục/Đào Tạo,"Kỹ năng giao tiếp, Kỹ năng giao tiếp, Kỹ năng ...","lô PT-01, KĐT Vinhomes Ocean Park, Đa Tốn, Gia...",Tốt nghiệp đại học trở lên. Khả năng sử dụng t...,Tổ chức/Thực hiện/Giám sát/Quản lý công việc S...,"Trên 40,000,000",1-3 năm,0.249874
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,"Giám Sát Nhà Phân Phối (Nghệ An, Hà Tĩnh)",Công Ty Cổ Phần Sữa Việt Nam (Vinamilk),Hàng tiêu dùng,"Kỹ năng giao tiếp, Kỹ năng nắm bắt và giải quy...","Hà Tĩnh, Việt Nam","HỌC VẤN & CHUYÊN NGÀNH:. Tốt nghiệp Đại học, ư...","KHU VỰC: Vinh, Quỳnh Lưu, Anh Sơn (Nghệ An) và...",Thương lượng,3-5 năm,0.252121
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Phiên Dịch Tiếng Nhật,Công Ty TNHH YKK Việt Nam,Dệt may/May mặc/Giày dép,"Kỹ năng giao tiêp, Kỹ năng Tin học văn phòng, ...","KCN Đồng Văn 3, Phường Hoàng Đông, Thị xã Duy ...",1. Bằng cấp: Tốt nghiệp Cao đẳng trở lên (chuy...,"MÔ TẢ CÔNG VIỆC. Làm các thủ tục, giấy tờ cho ...",Thương lượng,1-3 năm,0.255303
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Account Manager,Công Ty TNHH Truong Dinh Holding (Tdh),Điện/Điện tử,"Giao Tiếp Tốt, Tiếng anh giao tiếp tốt, Khả nă...","11B Nguyễn Bỉnh Khiêm, Phường Bến Nghé, Quận 1...",Tốt nghiệp Đại học trở lên chuyên ngành kỹ thu...,TÓM TẮT CÔNG VIỆC. Chịu trách nhiệm hỗ trợ hoạ...,Thương lượng,5-10 năm,0.260662
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Giám Sát Nhà Phân Phối - Tây Ninh,Công Ty Cổ Phần Sữa Việt Nam (Vinamilk),Hàng tiêu dùng,"Kỹ năng giao tiếp, Kỹ năng nắm bắt và giải quy...","Tây Ninh, Vietnam","HỌC VẤN & CHUYÊN NGÀNH:. Tốt nghiệp Đại học, ư...",Chịu trách nhiệm trước Trưởng bán hàng Vùn...,Thương lượng,3-5 năm,0.264940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Quản lý Khách hàng ưu tiên - Cà Mau,Ngân Hàng TMCP Quốc Tế Việt Nam (VIB),Ngân hàng,"Kỹ năng giao tiếp, Kỹ năng giao tiếp, Kỹ năng ...","Ngân hàng Quốc Tế - VIB Cà Mau, 206-208 đường ...",Tốt nghiệp Cao đẳng trở lên (đối với ứng viên ...,Trách nhiệm công việc:Chịu trách nhiệm (i) Tư ...,Thương lượng,1-3 năm,0.270114
196,973260,Lê Thanh,Vận tải - Lái xe,24,Chuyên Viên Đào Tạo & Phát Triển - Senior L&OD,Công Ty TNHH Livespo Pharma,Giáo dục/Đào Tạo,"Hợp Đồng Lao Động, Kỹ năng giao tiếp, Kỹ năng ...","Tầng 2, tòa N03-T5 Ngoại Giao Đoàn",Trình độ chuyên môn: Tốt nghiệp đại học các ng...,1. Lập kế hoạch và thực hiện chiến lược đào tạ...,"20,000,000 - 40,000,000",3-5 năm,0.271685
197,973260,Lê Thanh,Vận tải - Lái xe,24,[Jp-66] Kỹ Sư Điều Khiển Plc Cho Máy Gia Công Gỗ,Công Ty TNHH R Techno Việt Nam,Sản xuất,"Tiếng Nhật N3, Kinh nghiệm sửa chữa PLC, Kinh ...",Nhật Bản,JOB REQUIREMENTS（điều kiện cần thiết）. Tiếng N...,Công ty: Công ty SUZUKO. Nơi làm việc: Thành p...,"1,000,000 - 5,000,000",1-3 năm,0.271766
198,973260,Lê Thanh,Vận tải - Lái xe,24,Quản Lý Kinh Doanh Kênh OTC Dược phẩm Nhất Nhất,Công Ty TNHH Phân Phối Trực Tuyến Nhất Nhất,Dịch vụ Y tế/Chăm sóc sức khỏe,"Giao tiếp, Kỹ năng quản lý nhân sự, Kỹ năng qu...",Toà nhà Nhất Nhất Pharma - Số 20 lô 05 khu 4.1...,Từ 2 năm kinh nghiệm trở lên ở vị trí tương đư...,Tổng thu nhập: 30.000.000 40.000.000 đồng/thán...,"Trên 40,000,000",1-3 năm,0.273501


In [251]:
industry_recommendation_results = await Get_multi_recommendations(user_data, model, "phobert", "industry")
industry_recommendation_results

2024-12-16 01:28:35,431 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:28:35,434 INFO sqlalchemy.engine.Engine SELECT jobs_2.id, jobs_2.url, jobs_2.job_title, jobs_2.company_name, jobs_2.industry, jobs_2.company_address, jobs_2.job_description, jobs_2.job_requirements, jobs_2.benefits, jobs_2.salary, jobs_2.expire_date, jobs_2.employee_rank, jobs_2.job_skill, jobs_2.job_field, jobs_2.years_of_experience, jobs_2.job_address, jobs_2.job_title_vector, jobs_2.industry_vector, jobs_2.job_skill_vector, jobs_2.job_address_vector, jobs_2.industry_vector <=> $1 AS distance 
FROM jobs_2 
WHERE jobs_2.industry IS NOT NULL AND trim(regexp_replace(jobs_2.industry, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:28:35,435 INFO sqlalchemy.engine.Engine [cached since 1102s ago] ('[-0.06321824342012405,0.09771978855133057,-0.3152889609336853,-0.5855997800827026,-0.09530745446681976,-0.08576858043670654,0.27699097990989685,-0.39 ..

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Hành Chính (Lương Thoả Thuận Theo Nă...,Trung Tâm Kiểm Định Hiệu Chuẩn Vùng 3,Khoa học - Công nghệ,"Đào Tạo, Tư Vấn Bán Hàng, Hành chính Văn phòng","Đường ĐX17, Khu phố Phú Thọ, Phường Phú Chánh...",Nữ từ 24 40 tuổi;Tốt nghiệp Cao đẳng trở lên c...,"Tiếp nhận, tổng hợp số hóa hồ sơ hành chính th...","5,000,000 - 10,000,000",1-3 năm,0.332477
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Hành Chính (Lương Thoả Thuận Theo Nă...,Trung Tâm Kiểm Định Hiệu Chuẩn Vùng 3,Khoa học - Công nghệ,"Đào Tạo, Tư Vấn Bán Hàng, Hành chính Văn phòng","Đường ĐX17, Khu phố Phú Thọ, Phường Phú Chánh...",Nữ từ 24 40 tuổi;Tốt nghiệp Cao đẳng trở lên c...,"Tiếp nhận, tổng hợp số hóa hồ sơ hành chính th...","5,000,000 - 10,000,000",1-3 năm,0.332477
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Lập Trình Mobile (Flutter Dev),Công Ty TNHH Công Nghệ Katec,"Kinh doanh, Trang thiết bị văn phòng","Android, iOS, Lập trình Mobile, UI/UX","189 PHAN HUY CHÚ, AN KHÁNH, NINH KIỀU, TP CT,C...",Có ít nhất 1 năm kinh nghiệm làm việc về phát ...,Tham gia phát triển các dự án của công ty.Xây ...,"5,000,000 - 10,000,000",1-3 năm,0.362349
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Bán Hàng - Nhà Phân Phối Unilever,Công Ty TNHH Nguyên Minh Hoàng,"Kinh doanh, Bán hàng","Chăm Sóc Khách Hàng, Tư Vấn Bán Hàng, Bán Lẻ","K04/02 04/06 Ông Ích Khiêm, Tổ 56, Phường Than...",Nam/ Nữ(ưu tiên các bạn nam đã hoàn thành nghĩ...,Giới thiệu và bán các sản phẩm của Nhà sản xuấ...,Thỏa thuận,Không yêu cầu kinh nghiệm,0.369595
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Business Development Specialist (Job),Công Ty TNHH Chợ Tốt,"Kinh doanh, Bán hàng",Tư Vấn Doanh Nghiệp (B2B),"UOA Tower, Số 6 Tân Trào, Phường Tân Phú, Quận 7",Ít nhất 2 năm kinh nghiệm bán hàng/ tư vấn/ qu...,"Tìm kiếm, tiếp cận, giới thiệu, tư vấn và chốt...","5,000,000 - 10,000,000",1-3 năm,0.369595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Hà Nội - Chuyên Viên Kinh Doanh (Bán Xe Ô Tô V...,GSM,Vận tải,"Kinh doanh ô tô, Sales, Marketing","Tầng B1 tòa R3 Royal City, 72 Nguyễn Trãi, Tha...",Tốt nghiệp từ Trung cấp trở lên. Ít nhất 1 năm...,"Tìm kiếm Khách hàng, triển khai bán hàng, hoàn...","Trên 40,000,000",Không yêu cầu kinh nghiệm,0.282092
196,973260,Lê Thanh,Vận tải - Lái xe,24,Ruby On Rails Developer,GSM,Vận tải,"Ruby On Rails, OOP, MySQL, Python, PostgreSQL","Vincom Center Đồng Khởi, Đường Lê Thánh Tôn, B...",Required Skills and Qualifications:. 3+ years ...,"Key Responsibilities:. Develop, maintain, and ...","Trên 40,000,000",Không yêu cầu kinh nghiệm,0.282092
197,973260,Lê Thanh,Vận tải - Lái xe,24,Customs Clearance Manager,Samsung SDS,Vận tải,Electronics,"SEHC, Cổng số 1, Tăng Nhơn Phú B, Quận 9, Thàn...",(1)EDUCATION / QUALIFICATION:. College or univ...,(1) BROAD SCOPE:. Consulting of customs cleara...,"Trên 40,000,000",Trên 10 năm,0.282092
198,973260,Lê Thanh,Vận tải - Lái xe,24,Account Executive,"FedEx Express Vietnam Co., Ltd.",Vận tải,"Communications, Negotiation","15F, 4 Ton That Tung Street, Trung Tu Ward, Do...","To do the job well, you will need the followin...","In the position of an Account Executive, your ...",Thương lượng,1-3 năm,0.282092


In [252]:
address_recommendation_results = await Get_multi_recommendations(user_data, model, "phobert", "location")
address_recommendation_results

2024-12-16 01:28:40,735 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:28:40,741 INFO sqlalchemy.engine.Engine SELECT jobs_2.id, jobs_2.url, jobs_2.job_title, jobs_2.company_name, jobs_2.industry, jobs_2.company_address, jobs_2.job_description, jobs_2.job_requirements, jobs_2.benefits, jobs_2.salary, jobs_2.expire_date, jobs_2.employee_rank, jobs_2.job_skill, jobs_2.job_field, jobs_2.years_of_experience, jobs_2.job_address, jobs_2.job_title_vector, jobs_2.industry_vector, jobs_2.job_skill_vector, jobs_2.job_address_vector, jobs_2.job_address_vector <=> $1 AS distance 
FROM jobs_2 
WHERE jobs_2.job_address IS NOT NULL AND trim(regexp_replace(jobs_2.job_address, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:28:40,743 INFO sqlalchemy.engine.Engine [generated in 0.00307s] ('[-0.4315844178199768,0.06216150522232056,-0.555944561958313,-0.3563973903656006,-0.12172585725784302,0.03407549858093262,0.39096468687057495,-0.

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Phụ Trách Kinh Doanh,Công Ty TNHH Thép Tân Thắng Lợi,Kinh doanh,"Tư Vấn Bán Hàng, Phát Triển Thị Trường",Khánh Hòa,Tốt nghiệp cao đẳng trở lên chuyên ngành Thươn...,"Xây dựng, tìm các đối tác khách hàngBáo giá kh...","15,000,000 - 20,000,000",Không yêu cầu kinh nghiệm,0.000000
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Phó Quản Lý Chi Nhánh Thẩm Mỹ,Công Ty TNHH Dược Mỹ Phẩm H-Derma,Sản xuất/Vận hành sản xuất,"Mỹ Phẩm, Giám Sát Bán Hàng, Quản Lý Điều Hành",Khánh Hòa,Tối thiểu trên 01 năm kinh nghiệm làm vị trí t...,"Xây dựng, tổ chức, vận hành giám sát công việc...","20,000,000 - 40,000,000",Không yêu cầu kinh nghiệm,0.000000
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,New Product Introduction Engineer - NPI Engineer,Onsemi Viet Nam,Sản xuất,"Electronics, New Product Qualification, Lean S...",Biên Hòa,"Qualifications. BS/MS in Electrical, Mechanica...",Job Description. Introduces and maintains asse...,Thương lượng,Không yêu cầu kinh nghiệm,0.149836
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Tư Vấn Bán Hàng,Công Ty Cổ Phần Bán Lẻ Kỹ Thuật Số FPT,"Kinh doanh, Bán lẻ","Tư Vấn Bán Hàng, Bán Lẻ",Hòa Thành,"Nam/Nữ, tuổi từ 18 – 27.Tốt nghiệp THPT trở lê...",Chủ động đón tiếp khách hàng đến với của hàng....,"5,000,000 - 10,000,000",Không yêu cầu kinh nghiệm,0.212294
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Kinh Doanh,Công Ty TNHH Long Hưng Phát,"Bất động sản, Xây dựng","Tư Vấn Bán Hàng, Phát Triển Thị Trường, Nghiên...",Hòa Bình,Có 3 năm kinh nghiệm trong lĩnh vực kinh doanh...,"Tìm hiểu, khai thác và thu thập khách hàng tiề...","5,000,000 - 10,000,000",3-5 năm,0.225754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Kỹ Thuật IT Sửa PC Laptop Máy In,Công Ty TNHH Phát Triển Công Nghệ Trường Thịnh...,"Công nghệ thông tin, Quảng cáo/Marketing/PR","IT Helpdesk, IT Support",Hồ Chí Minh,Có Ít nhất > 1 năm KN làm chuyên viên kỹ thuật...,"Mua bán sửa chữa máy tính bàn pc, laptop, máy ...","5,000,000 - 10,000,000",1-3 năm,0.000000
196,973260,Lê Thanh,Vận tải - Lái xe,24,Quản Lý Kinh Doanh Thu Nhập Từ 20TR,Công Ty TNHH Bảo Hiểm Nhân Thọ Cathay - Việt Nam,bảo hiểm,"Chăm Sóc Khách Hàng, Tư Vấn Bán Hàng, Tư Vấn B...",Hồ Chí Minh,Tốt nghiệp trung cấp trở lên trên 26 tuổi. Đã ...,"Quản lý, đào tạo nhân viên, hỗ trợ tuyển dụng....","20,000,000 - 40,000,000",Không yêu cầu kinh nghiệm,0.000000
197,973260,Lê Thanh,Vận tải - Lái xe,24,Store Manager - Cửa Hàng Trưởng,Công ty TNHH Manpower Việt Nam - Hồ Chí Minh,Nhân Sự,"Quản Lý Điều Hành, Giám Sát Bán Hàng, Bán Lẻ",Hồ Chí Minh,At least 1 year of experience in fashion retai...,Stock & Sales Management:. Ensure effective re...,"20,000,000 - 40,000,000",1-3 năm,0.000000
198,973260,Lê Thanh,Vận tải - Lái xe,24,- Nhân Viên Phụ Bếp Part Time,Công ty Cổ phần Đầu tư Thương mại Quốc tế Mặt ...,Thực phẩm/DV ăn uống,"Phụ Bếp, Chế Biến Thực Phẩm, Tư Vấn Bán Hàng",Hồ Chí Minh,Yêu Cầu Công ViệcLàm việc tại: Nhà hàng Hotpot...,Thực hiện sơ chế các nguyên vật liệu dùng đ...,Thỏa thuận,Không yêu cầu kinh nghiệm,0.000000


In [253]:
job_title_recommendation_results = await Get_multi_recommendations(user_data, model, "phobert", "job title")
job_title_recommendation_results

2024-12-16 01:28:44,198 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:28:44,201 INFO sqlalchemy.engine.Engine SELECT jobs_2.id, jobs_2.url, jobs_2.job_title, jobs_2.company_name, jobs_2.industry, jobs_2.company_address, jobs_2.job_description, jobs_2.job_requirements, jobs_2.benefits, jobs_2.salary, jobs_2.expire_date, jobs_2.employee_rank, jobs_2.job_skill, jobs_2.job_field, jobs_2.years_of_experience, jobs_2.job_address, jobs_2.job_title_vector, jobs_2.industry_vector, jobs_2.job_skill_vector, jobs_2.job_address_vector, jobs_2.job_title_vector <=> $1 AS distance 
FROM jobs_2 
WHERE jobs_2.job_title IS NOT NULL AND trim(regexp_replace(jobs_2.job_title, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:28:44,202 INFO sqlalchemy.engine.Engine [generated in 0.00212s] ('[-0.10015001893043518,0.28205442428588867,-0.36834749579429626,-0.26571357250213623,-0.38623157143592834,0.29282432794570923,0.33689388632774353,-0.2 

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Hành Chính Văn Phòng (Tiếng Anh),Công Ty TNHH Gnf Japan,Kinh doanh,Hành chính Văn phòng,"Số nhà 9, Hẻm 03 Đường Nguyễn Trường Tộ, P Ea ...","Nam hoặc nữ có bằng đại họcThành thạo Word, Ex...",Các nhiệm vụ hành chính cơ bản và hỗ trợ kế to...,"5,000,000 - 10,000,000",1-3 năm,0.234521
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Hành Chính,Công ty TNHH HORN ( Việt Nam),Sản xuất/Vận hành sản xuất,"Hành chính Văn phòng, Lễ Tân (Receptionist), V...","An Dương, Hải Phòng",Nam/nữ: Tuổi từ 23. Tiếng Trung/ Tiếng Anh gia...,"Quản lý văn phòng phẩm và tài sản cố định, bao...",Thỏa thuận,1-3 năm,0.240540
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Hành Chính,Công Ty TNHH Thương Mại Dược Phẩm Y Tế Việt Hưng,Kinh doanh,"Quản Trị Nhân Sự (HR), Hành chính Văn phòng, V...","Số nhà 52, ngõ 1 Tập thể Trung Đoàn 17, Ngũ Hi...",Tốt nghiệp Cao đẳng trở lên các chuyên ngành: ...,Quản lí và lập kế hoạch tuyển dụng theo nhu cầ...,Thỏa thuận,1-3 năm,0.240540
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Hành Chính,Công Ty Cổ Phần Thép Đất Việt,Bán lẻ/Bán sỉ,"Sắp Xếp Lịch Hẹn, Đón Khách, Admin, Hành Chính...","Số 18 phố Quần Ngựa, Liễu Giai, Ba Đình, Hà Nội",Kinh nghiệm: Không yêu cầu. Yêu cầu bằng cấp: ...,Sắp xếp lịch và tiếp đón khách của Công ty. Tr...,Thương lượng,Không yêu cầu kinh nghiệm,0.240540
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Hành Chính,Công ty CPDV Hàng không Thăng Long,Trống,"Hành chính Văn phòng, Lễ Tân (Receptionist), V...",Hồ Chí Minh,"Yêu Cầu Công ViệcNam/ Nữ, 23 đến 30 tuổi.Tốt n...","Mô tả Công việcTheo dõi, quản lý mua sắm vật t...","10,000,000 - 15,000,000",Không yêu cầu kinh nghiệm,0.240540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Lái Xe,Trường Tiểu Học Nguyễn Siêu,Giáo dục/Đào Tạo,"Lái Xe Chuyên Nghiệp, Lái Xe, Quản Lý Xe, Bảo ...","Cầu Giấy, Hà Nội, Việt Nam",Yêu cầu ứng viên. Độ tuổi: Từ 25 đến 45 tuổi. ...,. Lái xe cho Chủ tịch Hội đồng Trường hàng ngà...,"Trên 40,000,000",Không yêu cầu kinh nghiệm,0.390532
196,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Lái Xe,Công Ty Cổ Phần Tập Đoàn Phú Minh Sơn,"Kinh doanh, Xây dựng","Lái Xe Tải, Vận Tải","7779 Hoàng Quốc Việt, Cầu Giấy, Hà Nội","Nam, tuổi từ 25 35. Bằng lái xe hạng B2. Có tố...","Thực hiện đưa đón, chuyên chở Ban Lãnh đạo và ...","5,000,000 - 10,000,000",Dưới 1 năm,0.390532
197,973260,Lê Thanh,Vận tải - Lái xe,24,"Nhân viên bán hàng Part-time, Thời vụ môi trườ...",Công Ty Cổ Phần Crystal Shine,Kinh Doanh,"Tư Vấn Bán Hàng, Bán Lẻ",Thủ Đức,*Nam nữ từ 1825 tuổi. *Có CCCD gốc. *Tốt nghiệ...,[ Shyne Store ] [ Tuyển dụng Nhân sự quận Thủ ...,"1,000,000 - 5,000,000",Dưới 1 năm,0.390721
198,973260,Lê Thanh,Vận tải - Lái xe,24,"Nhân viên bán hàng Part-time, Thời vụ môi trườ...",Công Ty Cổ Phần Crystal Shine,Kinh Doanh,"Tư Vấn Bán Hàng, Bán Lẻ",Quận 10,*Nam nữ từ 1825 tuổi. *Có CCCD gốc. *Tốt nghiệ...,[ Shyne Store ] [ Tuyển dụng Nhân sự quận 9 ] ...,"1,000,000 - 5,000,000",Dưới 1 năm,0.390721


In [254]:
combined_recommendation_results = await Get_multi_recommendations(user_data, model, "phobert", "combined")
combined_recommendation_results

2024-12-16 01:28:47,418 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:28:47,427 INFO sqlalchemy.engine.Engine SELECT jobs_2.id, jobs_2.url, jobs_2.job_title, jobs_2.company_name, jobs_2.industry, jobs_2.company_address, jobs_2.job_description, jobs_2.job_requirements, jobs_2.benefits, jobs_2.salary, jobs_2.expire_date, jobs_2.employee_rank, jobs_2.job_skill, jobs_2.job_field, jobs_2.years_of_experience, jobs_2.job_address, jobs_2.job_title_vector, jobs_2.industry_vector, jobs_2.job_skill_vector, jobs_2.job_address_vector, jobs_2.job_title_vector <=> $1 AS job_title_distance, jobs_2.industry_vector <=> $2 AS industry_distance, jobs_2.job_skill_vector <=> $3 AS job_skill_distance, jobs_2.job_address_vector <=> $4 AS job_address_distance 
FROM jobs_2 
WHERE jobs_2.industry IS NOT NULL AND jobs_2.job_title IS NOT NULL AND jobs_2.job_skill IS NOT NULL AND jobs_2.job_address IS NOT NULL AND trim(regexp_replace(jobs_2.industry, $5::VARCHAR, $6::VARCHAR, $7::VARCHAR)) != $8::VAR

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Hành Chính Văn Phòng (Tiếng Anh),Công Ty TNHH Gnf Japan,Kinh doanh,Hành chính Văn phòng,"Số nhà 9, Hẻm 03 Đường Nguyễn Trường Tộ, P Ea ...","Nam hoặc nữ có bằng đại họcThành thạo Word, Ex...",Các nhiệm vụ hành chính cơ bản và hỗ trợ kế to...,"5,000,000 - 10,000,000",1-3 năm,0.234521
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Hành Chính,Công Ty TNHH Thương Mại Dược Phẩm Y Tế Việt Hưng,Kinh doanh,"Quản Trị Nhân Sự (HR), Hành chính Văn phòng, V...","Số nhà 52, ngõ 1 Tập thể Trung Đoàn 17, Ngũ Hi...",Tốt nghiệp Cao đẳng trở lên các chuyên ngành: ...,Quản lí và lập kế hoạch tuyển dụng theo nhu cầ...,Thỏa thuận,1-3 năm,0.240540
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Hành Chính,Công Ty Cổ Phần Đầu Tư Quốc Tế - Mega Gangnam,Kinh doanh,Hành chính Văn phòng,"454 Xã Đàn, Nam Đồng, Đống Đa, Hà Nội","Nữ, ngoại hình sángTốt nghiệp Trung cấp trở lê...","Soạn thảo, lưu trữ, sắp xếp giấy tờ, hợp đồng,...","5,000,000 - 10,000,000",1-3 năm,0.240540
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Hành Chính,Công Ty Cổ Phần Tập Đoàn Công Nghiệp Việt,"Thương mại, Xây dựng xây lắp công nghiệp, Gia ...","Quản Trị Nhân Sự (HR), Hành chính Văn phòng, L...","Tầng 8 tòa nhà Silver Wings, 137A Nguyễn Văn C...",1. Kiến thức và kinh nghiệm. Trình độ học vấn:...,"1. Quản lý hồ sơ, giấy tờ:. Trực tiếp thực hiệ...","10,000,000 - 15,000,000",Dưới 1 năm,0.240540
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Hành Chính,Tập Đoàn Investcorp,Kỹ thuật xây dựng/Cơ sở hạ tầng,"Hành Chính, Truyền Thông, Truyền Thông Nội Bộ,...","Toà nhà INVESTCORP, Lô A15-16, Đường Vạn Phúc,...",Yêu cầu:. Tốt nghiệp đại học/cao đẳng. Ưu tiên...,Phụ trách công tác truyền thông của Tập đoàn: ...,Thương lượng,1-3 năm,0.240540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Lái Xe,Công Ty Cổ Phần Tập Đoàn Phú Minh Sơn,"Kinh doanh, Xây dựng","Lái Xe Tải, Vận Tải","7779 Hoàng Quốc Việt, Cầu Giấy, Hà Nội","Nam, tuổi từ 25 35. Bằng lái xe hạng B2. Có tố...","Thực hiện đưa đón, chuyên chở Ban Lãnh đạo và ...","5,000,000 - 10,000,000",Dưới 1 năm,0.390532
196,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Lái Xe,Công Ty Cổ Phần Vinhomes,"Kinh doanh, Bất động sản",Lái Xe,"Đảo Vũ Yên ,Huyện Thủy Nguyên, Thành phố Hải P...","Nam, có bằng lái xe C. Ưu tiên ứng viên có kin...","Lái xe tưới rửa đường, thu gom rác, quét đường...",Thỏa thuận,Không yêu cầu kinh nghiệm,0.390532
197,973260,Lê Thanh,Vận tải - Lái xe,24,"Nhân viên bán hàng Part-time, Thời vụ môi trườ...",Công Ty Cổ Phần Crystal Shine,Kinh Doanh,"Tư Vấn Bán Hàng, Bán Lẻ",Thủ Đức,*Nam nữ từ 1825 tuổi. *Có CCCD gốc. *Tốt nghiệ...,[ Shyne Store ] [ Tuyển dụng Nhân sự quận Thủ ...,"1,000,000 - 5,000,000",Dưới 1 năm,0.390721
198,973260,Lê Thanh,Vận tải - Lái xe,24,"Nhân viên bán hàng Part-time, Thời vụ môi trườ...",Công Ty Cổ Phần Crystal Shine,Kinh Doanh,"Tư Vấn Bán Hàng, Bán Lẻ",Bình Thạnh,*Nam nữ từ 1825 tuổi. *Có CCCD gốc. *Tốt nghiệ...,[ Shyne Store ] [ Tuyển dụng Nhân sự quận Bình...,"1,000,000 - 5,000,000",Dưới 1 năm,0.390721


In [255]:
skill_recommendation_results.to_csv("results/phobert/skill_recommendation_results.csv", index=False)
industry_recommendation_results.to_csv("results/phobert/industry_recommendation_results.csv", index=False)
address_recommendation_results.to_csv("results/phobert/address_recommendation_results.csv", index=False)
job_title_recommendation_results.to_csv("results/phobert/job_title_recommendation_results.csv", index=False)
combined_recommendation_results.to_csv("results/phobert/combined_recommendation_results.csv", index=False)

## XLM RoBERTa

In [256]:
model = SentenceTransformer('xlm-roberta-base')

No sentence-transformers model found with name xlm-roberta-base. Creating a new one with mean pooling.


In [257]:
skill_recommendation_results = await Get_multi_recommendations(user_data, model, "xlm_roberta", "skill")
skill_recommendation_results

2024-12-16 01:29:00,296 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:29:00,299 INFO sqlalchemy.engine.Engine SELECT jobs_3.id, jobs_3.url, jobs_3.job_title, jobs_3.company_name, jobs_3.industry, jobs_3.company_address, jobs_3.job_description, jobs_3.job_requirements, jobs_3.benefits, jobs_3.salary, jobs_3.expire_date, jobs_3.employee_rank, jobs_3.job_skill, jobs_3.job_field, jobs_3.years_of_experience, jobs_3.job_address, jobs_3.job_title_vector, jobs_3.industry_vector, jobs_3.job_skill_vector, jobs_3.job_address_vector, jobs_3.job_skill_vector <=> $1 AS distance 
FROM jobs_3 
WHERE jobs_3.job_skill IS NOT NULL AND trim(regexp_replace(jobs_3.job_skill, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:29:00,301 INFO sqlalchemy.engine.Engine [cached since 1957s ago] ('[-0.018804701045155525,0.015157408080995083,0.025009622797369957,0.02211725153028965,0.14088299870491028,0.023517217487096786,-0.019457638263702393,0

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,"Giám Sát Nhà Phân Phối (Nghệ An, Hà Tĩnh)",Công Ty Cổ Phần Sữa Việt Nam (Vinamilk),Hàng tiêu dùng,"Kỹ năng giao tiếp, Kỹ năng nắm bắt và giải quy...","Hà Tĩnh, Việt Nam","HỌC VẤN & CHUYÊN NGÀNH:. Tốt nghiệp Đại học, ư...","KHU VỰC: Vinh, Quỳnh Lưu, Anh Sơn (Nghệ An) và...",Thương lượng,3-5 năm,0.002492
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Giám Sát Nhà Phân Phối - Tây Ninh,Công Ty Cổ Phần Sữa Việt Nam (Vinamilk),Hàng tiêu dùng,"Kỹ năng giao tiếp, Kỹ năng nắm bắt và giải quy...","Tây Ninh, Vietnam","HỌC VẤN & CHUYÊN NGÀNH:. Tốt nghiệp Đại học, ư...",Chịu trách nhiệm trước Trưởng bán hàng Vùn...,Thương lượng,3-5 năm,0.002525
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Kinh Doanh MT (Toàn Quốc),Stavian Group,Tài Chính,"Kỹ năng bán hàng, Kỹ năng bán hàng, Kỹ năng gi...","12A Vincom Center, 72 Lê Thánh Tôn, phường Bến...","1. Kiến thức: Kinh doanh, tài chính, chăm sóc ...",1. Mục tiêu: Xây dựng và phát triển hệ thống k...,Thương lượng,5-10 năm,0.002581
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Giám sát Nhà phân phối - Bà Rịa Vũng Tàu,Công Ty Cổ Phần Sữa Việt Nam (Vinamilk),Hàng tiêu dùng,"Kỹ năng giao tiếp, Kỹ năng giao tiếp, Kỹ năng ...",Bà Rịa - Vũng Tàu,"HỌC VẤN & CHUYÊN NGÀNH:. Tốt nghiệp Đại học, ư...",Chịu trách nhiệm trước Trưởng bán hàng Vùn...,Thương lượng,3-5 năm,0.002632
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Nhóm Sự Kiện,Hệ thống Giáo Dục Edufit,Giáo dục/Đào Tạo,"Kỹ năng giao tiếp, Kỹ năng giao tiếp, Kỹ năng ...","lô PT-01, KĐT Vinhomes Ocean Park, Đa Tốn, Gia...",Tốt nghiệp đại học trở lên. Khả năng sử dụng t...,Tổ chức/Thực hiện/Giám sát/Quản lý công việc S...,"Trên 40,000,000",1-3 năm,0.002678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Chuyên Viên Tuyển Dụng (Mảng Nội thất - Lương ...,Công Ty TNHH PNK,Nội thất/Gỗ,"Kỹ năng giao tiếp, Kỹ năng giao tiếp, Kỹ năng ...","Showroom – KĐT Gamuda, phường Trần Phú, quận H...",Tốt nghiệp từ Cao Đẳng trở lên. Có từ 03 năm k...,"Lập kế hoạch tuyển dụng, ngân sách tuyển dụng ...","10,000,000 - 15,000,000",3-5 năm,0.003876
196,973260,Lê Thanh,Vận tải - Lái xe,24,Business Development Manager (Attractive Salary),Công Ty Cổ Phần Seadent,Dịch vụ Y tế/Chăm sóc sức khỏe,"Kỹ năng lập kế hoạch, Kỹ năng tư duy dịch vụ, ...","13 Đặng Tất, Tân Định, Quận 1, Thành phố Hồ Ch...",JOB REQUIREMENTS. Bachelor’s Degree in Busines...,1. Product Management:. Develop and manage the...,Thương lượng,3-5 năm,0.003912
197,973260,Lê Thanh,Vận tải - Lái xe,24,Admin Dự Án,CÔNG TY TNHH MỘT THÀNH VIÊN KIẾN TRÚC LOGIC,Kiến trúc/Thiết kế nội thất,"Kỹ năng giao tiêp, Kỹ năng đàm phán, Kỹ năng đ...","02 Trương Quyền, P.Võ Thị Sáu, Quận 3, TP.HCM",BẮT BUỘC. Có kinh nghiệm ở vị trí thư ký dự án...,"Làm hồ sơ thanh toán, quyết toán, của dự án vớ...","5,000,000 - 10,000,000",Không yêu cầu kinh nghiệm,0.003967
198,973260,Lê Thanh,Vận tải - Lái xe,24,Trưởng Phòng Kinh Doanh Kỹ Thuật Dự Án (Khu vự...,Công Ty TNHH Điều Hòa Gree (Việt Nam),Điện/Điện tử,"Chịu trách nhiệm công việc, Kinh doanh dự án, ...","Tầng 9, tòa nhà Bạch Đằng, số 268 Trần Nguyên ...",Có kinh nghiệm làm việc ở vị trí điều hòa thươ...,Có kinh nghiệm làm việc dự án. Có mối quan hệ ...,"15,000,000 - 20,000,000",3-5 năm,0.003978


In [258]:
address_recommendation_results = await Get_multi_recommendations(user_data, model, "xlm_roberta", "location")
address_recommendation_results

2024-12-16 01:29:03,961 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:29:03,963 INFO sqlalchemy.engine.Engine SELECT jobs_3.id, jobs_3.url, jobs_3.job_title, jobs_3.company_name, jobs_3.industry, jobs_3.company_address, jobs_3.job_description, jobs_3.job_requirements, jobs_3.benefits, jobs_3.salary, jobs_3.expire_date, jobs_3.employee_rank, jobs_3.job_skill, jobs_3.job_field, jobs_3.years_of_experience, jobs_3.job_address, jobs_3.job_title_vector, jobs_3.industry_vector, jobs_3.job_skill_vector, jobs_3.job_address_vector, jobs_3.job_address_vector <=> $1 AS distance 
FROM jobs_3 
WHERE jobs_3.job_address IS NOT NULL AND trim(regexp_replace(jobs_3.job_address, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:29:03,965 INFO sqlalchemy.engine.Engine [cached since 1943s ago] ('[0.011592311784625053,0.09437571465969086,0.029185447841882706,-0.0574013814330101,0.09281326830387115,0.058053914457559586,-0.0103012146428227

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Phó Quản Lý Chi Nhánh Thẩm Mỹ,Công Ty TNHH Dược Mỹ Phẩm H-Derma,Sản xuất/Vận hành sản xuất,"Mỹ Phẩm, Giám Sát Bán Hàng, Quản Lý Điều Hành",Khánh Hòa,Tối thiểu trên 01 năm kinh nghiệm làm vị trí t...,"Xây dựng, tổ chức, vận hành giám sát công việc...","20,000,000 - 40,000,000",Không yêu cầu kinh nghiệm,0.000000
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Phụ Trách Kinh Doanh,Công Ty TNHH Thép Tân Thắng Lợi,Kinh doanh,"Tư Vấn Bán Hàng, Phát Triển Thị Trường",Khánh Hòa,Tốt nghiệp cao đẳng trở lên chuyên ngành Thươn...,"Xây dựng, tìm các đối tác khách hàngBáo giá kh...","15,000,000 - 20,000,000",Không yêu cầu kinh nghiệm,0.000000
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,New Product Introduction Engineer - NPI Engineer,Onsemi Viet Nam,Sản xuất,"Electronics, New Product Qualification, Lean S...",Biên Hòa,"Qualifications. BS/MS in Electrical, Mechanica...",Job Description. Introduces and maintains asse...,Thương lượng,Không yêu cầu kinh nghiệm,0.001145
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Phát Triển Thị Trường Thức Ăn Thủy S...,Công Ty TNHH Thương Mại - Dịch Vụ Diên Khánh,Kinh doanh/Thủy Sản/Nông nghiệp,"Tư Vấn Bán Hàng, Phát Triển Thị Trường",Ninh Thuận,"Độ tuổi 2240;Đã tốt nghiệp Trung cấp, Cao đẳng...","Bán hàng, chăm sóc hệ thống đại lý cũPhát triể...","10,000,000 - 15,000,000",Dưới 1 năm,0.001607
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Phát Triển Thị Trường Thức Ăn Thủy S...,Công Ty TNHH Thương Mại - Dịch Vụ Diên Khánh,Kinh doanh/Thủy Sản/Nông nghiệp,"Tư Vấn Bán Hàng, Phát Triển Thị Trường",Ninh Thuận,"Độ tuổi 2240;Đã tốt nghiệp Trung cấp, Cao đẳng...","Bán hàng, chăm sóc hệ thống đại lý cũPhát triể...","10,000,000 - 15,000,000",Dưới 1 năm,0.001607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Trưởng Phòng Kiểm Soát Chất Lượng,Công Ty TNHH In Bao Bì Toàn Phát,Sản xuất/Vận hành sản xuất,"Vận Hành Sản Xuất, Quản Lý Chất Lượng (QA/QC),...",Hồ Chí Minh,Độ tuổi: dưới 45. Tốt nghiệp đại học trở lên. ...,Xây dựng quy trình và đội ngũ về chất lượng ng...,Thỏa thuận,1-3 năm,0.000000
196,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Kinh Doanh Dự Án Thiết Bị Điện,Công ty TNHH Etinco,Kinh doanh,Tư Vấn Bán Hàng,Hồ Chí Minh,Có kinh nghiệm từ 3 năm trong lĩnh vực kinh do...,"Tìm hiểu, nghiên cứu và nắm vững sản phẩm thươ...","15,000,000 - 20,000,000",3-5 năm,0.000000
197,973260,Lê Thanh,Vận tải - Lái xe,24,Chuyên Viên Điều Tra An Ninh,Công Ty Tài chính TNHH HD SAISON,Kinh doanh,"Phân Tích Tài Chính, Tâm lý học, Luật",Hồ Chí Minh,"1. Trình độ học vấn:Sơ cấp,Trung cấp, Đại học ...",Tiến hành điều tra độc lập đối với các vụ việc...,Thỏa thuận,1-3 năm,0.000000
198,973260,Lê Thanh,Vận tải - Lái xe,24,Kỹ Thuật IT Sửa PC Laptop Máy In,Công Ty TNHH Phát Triển Công Nghệ Trường Thịnh...,"Công nghệ thông tin, Quảng cáo/Marketing/PR","IT Helpdesk, IT Support",Hồ Chí Minh,Có Ít nhất > 1 năm KN làm chuyên viên kỹ thuật...,"Mua bán sửa chữa máy tính bàn pc, laptop, máy ...","5,000,000 - 10,000,000",1-3 năm,0.000000


In [259]:
industry_recommendation_results = await Get_multi_recommendations(user_data, model, "xlm_roberta", "industry")
industry_recommendation_results

2024-12-16 01:29:06,793 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:29:06,799 INFO sqlalchemy.engine.Engine SELECT jobs_3.id, jobs_3.url, jobs_3.job_title, jobs_3.company_name, jobs_3.industry, jobs_3.company_address, jobs_3.job_description, jobs_3.job_requirements, jobs_3.benefits, jobs_3.salary, jobs_3.expire_date, jobs_3.employee_rank, jobs_3.job_skill, jobs_3.job_field, jobs_3.years_of_experience, jobs_3.job_address, jobs_3.job_title_vector, jobs_3.industry_vector, jobs_3.job_skill_vector, jobs_3.job_address_vector, jobs_3.industry_vector <=> $1 AS distance 
FROM jobs_3 
WHERE jobs_3.industry IS NOT NULL AND trim(regexp_replace(jobs_3.industry, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:29:06,799 INFO sqlalchemy.engine.Engine [cached since 1957s ago] ('[0.024326207116246223,0.07340653985738754,-0.003921078983694315,0.04226132482290268,0.08568193763494492,-0.04365711659193039,-0.029844509437680244,0. ..

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trình Dược Viên Phòng Khám Khu Vực TP HCM – Hà...,Công Ty Cổ phần Dược Phẩm Việt Hà,"Y tế, Bán hàng","Tư Vấn Dược Phẩm, Tư Vấn Bán Hàng, Giám Sát Bá...",Ba Đình,"Tốt nghiệp trung cấp dược, ưu tiên có thêm chu...",Giới thiệu và bán sản phẩm của cty đến hệ thốn...,"10,000,000 - 15,000,000",1-3 năm,0.003510
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Chuyên Viên Chăm Sóc Khách Hàng,De La Sól By SunLife Vietnam,Bảo hiểm/ Tài chính,"Tổng Đài Viên, IT Helpdesk, Chăm Sóc Khách Hàn...","29 Liễu Giai, Ngọc Khánh, Ba Đình, Hà Nội 100000","Tốt nghiệp CĐ / ĐH ,từ 24 tuổi trở lên. Có kỹ ...","Tìm kiếm, mở rộng và xây dựng nguồn khách hàng...","15,000,000 - 20,000,000",1-3 năm,0.003775
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Chuyên Viên Quản Lý Khách Hàng Cấp Cao (Thu Nh...,De La Sól By SunLife Vietnam,Bảo hiểm/ Tài chính,Quan Hệ Khách Hàng,"Tòa nhà Hồng Hà, 25 Lý Thường Kiệt, Phường Pha...",Tốt nghiệp CĐ / ĐH (Ưu tiên ứng viên học các c...,Tư vấn tài chính :. Cung cấp và hỗ trợ các giả...,"15,000,000 - 20,000,000",1-3 năm,0.003775
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Center Director (Giám Đốc Phát Triển Kinh Doanh),De La Sól By SunLife Vietnam,Bảo hiểm/ Tài chính,"Phân Tích Tài Chính, Quản Lý Tín Dụng, Quan Hệ...","29 Liễu Giai, Ngọc Khánh, Ba Đình, Hà Nội 100000",Good English in advantage (Writing & Speaking)...,The incumbent of this job is responsible for d...,"20,000,000 - 40,000,000",1-3 năm,0.003775
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Lái Xe,Công Ty Cổ Phần Tập Đoàn Phú Minh Sơn,"Kinh doanh, Xây dựng","Lái Xe Tải, Vận Tải","7779 Hoàng Quốc Việt, Cầu Giấy, Hà Nội","Nam, tuổi từ 25 35. Bằng lái xe hạng B2. Có tố...","Thực hiện đưa đón, chuyên chở Ban Lãnh đạo và ...","5,000,000 - 10,000,000",Dưới 1 năm,0.003777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Kho Kiêm Giao Nhận Xe Máy,Công Ty TNHH DV Giao Nhận vận Tải Quốc Tế Thái...,Giao thông vận tải/Thủy lợi,"Thủ Kho, Giao Nhận Hàng, Giám Sát Kho","119 Đào Duy Anh, Phường 09, Quận Phú Nhuận, Th...","Tuổi dưới 40, có sức khỏe tốtNhiệt tình, siêng...",Thực hiện các công việc ở kho: chia chọn sắp x...,Thỏa thuận,Không yêu cầu kinh nghiệm,0.002606
196,973260,Lê Thanh,Vận tải - Lái xe,24,General Accountant Supervisor,Công ty TNHH Chuyển phát nhanh Yamato và 365,Giao thông vận tải/Thủy lợi,"Kế Toán Tổng Hợp, Kiểm Toán, CPA","14th Floor, Handico Tower, Pham Hung Street, M...",Good at English all 4 skillAbility to work und...,Record input expenses in the accounting system...,"20,000,000 - 40,000,000",3-5 năm,0.002606
197,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Kinh Doanh Logistics,Công Ty TNHH AT Á Châu,Giao thông vận tải/Thủy lợi,"Tư Vấn Bán Hàng, Xuất Nhập Khẩu","Thái Hà, Đống Đa","Tốt nghiệp đại học ngành Logistics, Kinh tế , ...","Tìm kiếm, chào bán, tư vấn chuỗi cung ứng, dịc...",Thỏa thuận,1-3 năm,0.002606
198,973260,Lê Thanh,Vận tải - Lái xe,24,Trợ Lý Giám Đốc Kinh Doanh,Công Ty TNHH Icel Logistics,Giao thông vận tải/Thủy lợi,"Vận Hành Sản Xuất, Biên/Phiên Dịch, Thư Ký, Hà...","Tầng 12, Tháp A, Tòa nhà Handiresco, Số 521 Ki...","Ưu tiên có kiến thức cơ bản về logistics, xuất...","Phối hợp cùng các phòng ban để tiếp nhận, xử l...",Thỏa thuận,Không yêu cầu kinh nghiệm,0.002606


In [260]:
job_title_recommendation_results = await Get_multi_recommendations(user_data, model, "xlm_roberta", "job title")
job_title_recommendation_results

2024-12-16 01:29:09,475 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:29:09,477 INFO sqlalchemy.engine.Engine SELECT jobs_3.id, jobs_3.url, jobs_3.job_title, jobs_3.company_name, jobs_3.industry, jobs_3.company_address, jobs_3.job_description, jobs_3.job_requirements, jobs_3.benefits, jobs_3.salary, jobs_3.expire_date, jobs_3.employee_rank, jobs_3.job_skill, jobs_3.job_field, jobs_3.years_of_experience, jobs_3.job_address, jobs_3.job_title_vector, jobs_3.industry_vector, jobs_3.job_skill_vector, jobs_3.job_address_vector, jobs_3.job_title_vector <=> $1 AS distance 
FROM jobs_3 
WHERE jobs_3.job_title IS NOT NULL AND trim(regexp_replace(jobs_3.job_title, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:29:09,478 INFO sqlalchemy.engine.Engine [cached since 1943s ago] ('[-0.018961165100336075,0.05322413891553879,-0.006321473978459835,0.05661774054169655,0.057976894080638885,0.01587197370827198,-0.012791955843567848,0

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân viên tư vấn - bán hàng,Công Ty TNHH Thương Mại Dịch Vụ Công Nghệ Dược...,kinh doanh,"Tư Vấn Bán Hàng, Bán Lẻ","quận Gò Vấp, Hồ Chí Minh","Nam/Nữ Từ 18 – 35 tuổi, có đam mê và nhiệt huy...","Tư vấn khách hàng, giới thiệu sản phẩm của côn...","1,000,000 - 5,000,000",Không yêu cầu kinh nghiệm,0.002167
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Cộng tác viên Nhập liệu tại văn phòng,Công ty CP Công nghệ Phần mềm và Nội dung số OSP,Công nghệ thông tin,"Nhập Liệu, Văn Thư/Lưu Trữ, Thư Viện","Tầng 4, Tòa Garden Hill, Số 99 Trần Bình",Kỹ năng chuyên môn hoặc kỹ năng mềm cần thiết ...,Nhập dữ liệu hộ tịch trên excelĐược hướng dẫn ...,Thỏa thuận,Không yêu cầu kinh nghiệm,0.002169
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân viên chăm sóc khách hàng,Công Ty Cổ Phần VFL INVEST,Bất động sản,Bán Hàng Qua Điện Thoại (Telesale),"169 Nguyễn Ngọc Vũ, Trung Hoà, Cầu Giấy, Hà Nội","Không yêu cầu kinh nghiệm, chưa có sẽ được cầm...",Gọi điện tìm kiếm khách hàng từ những nguồn da...,"5,000,000 - 10,000,000",Không yêu cầu kinh nghiệm,0.002355
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân viên đóng gói theo giờ,Công Ty Cổ Phần Gbc Global,Kinh doanh,"Phụ Kho, Công Nhân","Biệt thự Khu đô thị Văn Phú, Hà Đông, Hà Nội","Cẩn thận, tỉ mỉ, trung thực và có trách nhiệm ...",Đóng gói giỏ hàng Tết. Thực hiện công việc đón...,Thỏa thuận,Không yêu cầu kinh nghiệm,0.002609
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân viên tư vấn bán hàng part time,Công Ty TNHH Thương Mại Dịch Vụ Công Nghệ Dược...,kinh doanh,"Tư Vấn Bán Hàng, Bán Lẻ","quận Gò Vấp, Hồ Chí Minh","Nam/Nữ Từ 18 – 35 tuổi, có đam mê và nhiệt huy...","Tư vấn khách hàng, giới thiệu sản phẩm của côn...","1,000,000 - 5,000,000",Không yêu cầu kinh nghiệm,0.002765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Trưởng Phòng mua hàng nội địa (Công ty Cơ điện),Công Ty Cổ Phần T.T.S.C,Cơ khí/Máy móc/Thiết bị công nghiệp,"Quản Lý, Theo Dõi Đơn Hàng, Purchasing, Cung Ứ...",VP 219 Trung Kính - Cầu Giấy - Hà Nội,Ứng viên có tối thiểu 2 năm làm việc tại vị tr...,"Điều hành, quản lý nhân viên trong phòng thực ...","15,000,000 - 20,000,000",1-3 năm,0.003386
196,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Setting Máy CNC - Làm việc tại Thái ...,Công ty Cổ Phần Hyundai Aluminum vina,Sản xuất,"Chạy máy CNC, Làm Việc Theo Quy Trình, Lập Trì...","Khu Công Nghiệp Điềm Thụy Thái Nguyên, Phú Bìn...","Nam, tốt nghiệp Trung cấp, Cao đẳng hoặc Đại h...","Thực hiện cài đặt và vận hành máy CNC, đảm bảo...","Trên 40,000,000",1-3 năm,0.003408
197,973260,Lê Thanh,Vận tải - Lái xe,24,"Nhân viên bán hàng tại Tân Bình Part-time, Thờ...",Công Ty Cổ Phần Crystal Shine,Kinh Doanh,"Tư Vấn Bán Hàng, Bán Lẻ",Tân Bình,*Nam nữ từ 1825 tuổi. *Có CCCD gốc. *Tốt nghiệ...,[ Shyne Store ] [ Tuyển dụng ] Nhân viên bán h...,"1,000,000 - 5,000,000",Dưới 1 năm,0.003461
198,973260,Lê Thanh,Vận tải - Lái xe,24,Thực tập sinh tuyển dụng (Mạnh về thiết kế),Công ty Cổ phần Dược phẩm ECO,Y tế,"Tuyển Dụng (HR), Hành chính Văn phòng","198 Hoàng Văn Thụ, P. 9, Quận Phú Nhuận","Ngoại hình dễ nhìn, năng động;. Có laptop cá n...",Phát triển thương hiệu tuyển dụng ( thiết kế c...,"1,000,000 - 5,000,000",Không yêu cầu kinh nghiệm,0.003495


In [261]:
combined_recommendation_results = await Get_multi_recommendations(user_data, model, "xlm_roberta", "combined")
combined_recommendation_results

2024-12-16 01:29:12,463 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:29:12,470 INFO sqlalchemy.engine.Engine SELECT jobs_3.id, jobs_3.url, jobs_3.job_title, jobs_3.company_name, jobs_3.industry, jobs_3.company_address, jobs_3.job_description, jobs_3.job_requirements, jobs_3.benefits, jobs_3.salary, jobs_3.expire_date, jobs_3.employee_rank, jobs_3.job_skill, jobs_3.job_field, jobs_3.years_of_experience, jobs_3.job_address, jobs_3.job_title_vector, jobs_3.industry_vector, jobs_3.job_skill_vector, jobs_3.job_address_vector, jobs_3.job_title_vector <=> $1 AS job_title_distance, jobs_3.industry_vector <=> $2 AS industry_distance, jobs_3.job_skill_vector <=> $3 AS job_skill_distance, jobs_3.job_address_vector <=> $4 AS job_address_distance 
FROM jobs_3 
WHERE jobs_3.industry IS NOT NULL AND jobs_3.job_title IS NOT NULL AND jobs_3.job_skill IS NOT NULL AND jobs_3.job_address IS NOT NULL AND trim(regexp_replace(jobs_3.industry, $5::VARCHAR, $6::VARCHAR, $7::VARCHAR)) != $8::VAR

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân viên tư vấn - bán hàng,Công Ty TNHH Thương Mại Dịch Vụ Công Nghệ Dược...,kinh doanh,"Tư Vấn Bán Hàng, Bán Lẻ","quận Gò Vấp, Hồ Chí Minh","Nam/Nữ Từ 18 – 35 tuổi, có đam mê và nhiệt huy...","Tư vấn khách hàng, giới thiệu sản phẩm của côn...","1,000,000 - 5,000,000",Không yêu cầu kinh nghiệm,0.002167
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Cộng tác viên Nhập liệu tại văn phòng,Công ty CP Công nghệ Phần mềm và Nội dung số OSP,Công nghệ thông tin,"Nhập Liệu, Văn Thư/Lưu Trữ, Thư Viện","Tầng 4, Tòa Garden Hill, Số 99 Trần Bình",Kỹ năng chuyên môn hoặc kỹ năng mềm cần thiết ...,Nhập dữ liệu hộ tịch trên excelĐược hướng dẫn ...,Thỏa thuận,Không yêu cầu kinh nghiệm,0.002169
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân viên chăm sóc khách hàng,Công Ty Cổ Phần VFL INVEST,Bất động sản,Bán Hàng Qua Điện Thoại (Telesale),"169 Nguyễn Ngọc Vũ, Trung Hoà, Cầu Giấy, Hà Nội","Không yêu cầu kinh nghiệm, chưa có sẽ được cầm...",Gọi điện tìm kiếm khách hàng từ những nguồn da...,"5,000,000 - 10,000,000",Không yêu cầu kinh nghiệm,0.002355
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân viên đóng gói theo giờ,Công Ty Cổ Phần Gbc Global,Kinh doanh,"Phụ Kho, Công Nhân","Biệt thự Khu đô thị Văn Phú, Hà Đông, Hà Nội","Cẩn thận, tỉ mỉ, trung thực và có trách nhiệm ...",Đóng gói giỏ hàng Tết. Thực hiện công việc đón...,Thỏa thuận,Không yêu cầu kinh nghiệm,0.002609
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân viên tư vấn bán hàng part time,Công Ty TNHH Thương Mại Dịch Vụ Công Nghệ Dược...,kinh doanh,"Tư Vấn Bán Hàng, Bán Lẻ","quận Gò Vấp, Hồ Chí Minh","Nam/Nữ Từ 18 – 35 tuổi, có đam mê và nhiệt huy...","Tư vấn khách hàng, giới thiệu sản phẩm của côn...","1,000,000 - 5,000,000",Không yêu cầu kinh nghiệm,0.002765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Trưởng Phòng mua hàng nội địa (Công ty Cơ điện),Công Ty Cổ Phần T.T.S.C,Cơ khí/Máy móc/Thiết bị công nghiệp,"Quản Lý, Theo Dõi Đơn Hàng, Purchasing, Cung Ứ...",VP 219 Trung Kính - Cầu Giấy - Hà Nội,Ứng viên có tối thiểu 2 năm làm việc tại vị tr...,"Điều hành, quản lý nhân viên trong phòng thực ...","15,000,000 - 20,000,000",1-3 năm,0.003386
196,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Setting Máy CNC - Làm việc tại Thái ...,Công ty Cổ Phần Hyundai Aluminum vina,Sản xuất,"Chạy máy CNC, Làm Việc Theo Quy Trình, Lập Trì...","Khu Công Nghiệp Điềm Thụy Thái Nguyên, Phú Bìn...","Nam, tốt nghiệp Trung cấp, Cao đẳng hoặc Đại h...","Thực hiện cài đặt và vận hành máy CNC, đảm bảo...","Trên 40,000,000",1-3 năm,0.003408
197,973260,Lê Thanh,Vận tải - Lái xe,24,"Nhân viên bán hàng tại Tân Bình Part-time, Thờ...",Công Ty Cổ Phần Crystal Shine,Kinh Doanh,"Tư Vấn Bán Hàng, Bán Lẻ",Tân Bình,*Nam nữ từ 1825 tuổi. *Có CCCD gốc. *Tốt nghiệ...,[ Shyne Store ] [ Tuyển dụng ] Nhân viên bán h...,"1,000,000 - 5,000,000",Dưới 1 năm,0.003461
198,973260,Lê Thanh,Vận tải - Lái xe,24,Thực tập sinh tuyển dụng (Mạnh về thiết kế),Công ty Cổ phần Dược phẩm ECO,Y tế,"Tuyển Dụng (HR), Hành chính Văn phòng","198 Hoàng Văn Thụ, P. 9, Quận Phú Nhuận","Ngoại hình dễ nhìn, năng động;. Có laptop cá n...",Phát triển thương hiệu tuyển dụng ( thiết kế c...,"1,000,000 - 5,000,000",Không yêu cầu kinh nghiệm,0.003495


In [262]:
skill_recommendation_results.to_csv("results/xlm_roberta/skill_recommendation_results.csv", index=False)
industry_recommendation_results.to_csv("results/xlm_roberta/industry_recommendation_results.csv", index=False)
address_recommendation_results.to_csv("results/xlm_roberta/address_recommendation_results.csv", index=False)
job_title_recommendation_results.to_csv("results/xlm_roberta/job_title_recommendation_results.csv", index=False)
combined_recommendation_results.to_csv("results/xlm_roberta/combined_recommendation_results.csv", index=False)